# <center>**[프로젝트] 법률 문서 기반 검색 에이전트** </center>


---

### **Adaptive RAG**

- Adaptive RAG는 (1) 쿼리 분석과 (2) 능동적/자기 수정적 RAG를 결합한 전략

- 질문의 특성에 따라 가장 적합한 검색 및 응답 방식을 선택

- 논문에서의 라우팅 전략:
    1. No Retrieval: 검색 없이 직접 응답 
    1. Single-shot RAG: 단일 검색 후 응답 
    1. Iterative RAG: 반복적인 검색과 응답 과정 

- 논문: https://arxiv.org/abs/2403.14403


---

### **Self-RAG**

- Self-RAG (Retrieval-Augmented Generation with Self-Reflection) 

- 주요 단계: 

   1. 검색 결정 (Retrieval Decision):
      - 입력: 질문 `x` 또는 질문 `x`와 생성된 답변 `y`
      - 목적: 검색기 `R`을 사용하여 `D` 개의 청크를 검색할지 결정
      - 출력: "yes", "no", "continue" 중 하나
      - 의미: 시스템이 추가 정보가 필요한지 판단

   2. 검색된 문서 관련성 평가:
      - 입력: 질문 `x`와 각 검색된 청크 `d`
      - 목적: 각 청크가 질문에 유용한 정보를 제공하는지 평가
      - 출력: "relevant" 또는 "irrelevant"
      - 의미: 관련 없는 정보를 필터링하여 품질을 향상

   3. 생성된 답변의 환각 평가:
      - 입력: 질문 `x`, 청크 `d`, 생성된 텍스트 `y`
      - 목적: 생성된 텍스트가 청크의 정보에 의해 지지되는지 평가
      - 출력: "fully supported", "partially supported", "no support"
      - 의미: 환각(hallucination)을 감지하고 정보의 신뢰성을 확인

   4. 생성된 답변의 유용성 평가:
      - 입력: 질문 `x`와 생성된 텍스트 `y`
      - 목적: 생성된 텍스트가 질문에 유용한 응답인지 평가
      - 출력: 5점 척도 (5: 매우 유용, 1: 전혀 유용하지 않음)
      - 의미: 응답의 품질과 관련성을 수치화

- 논문: https://arxiv.org/abs/2310.11511

---

### **Corrective RAG (CRAG)**

- CRAG (Corrective Retrieval-Augmented Generation) 

- 주요 과정: 검색 -> 평가 -> 지식 정제 또는 웹 검색 -> 답변 생성

   1. 문서 관련성 평가 (`grade_documents`):
      - 각 문서의 관련성을 평가
      - 기준을 통과하는 문서만을 유지

   1. 지식 정제 (`refine_knowledge`):
      - 문서를 "지식 조각"으로 분할하고 각각의 관련성을 평가
      - 관련성 높은(0.5 초과) 지식 조각만 유지

   1. 웹 검색 (`web_search`):
      - 문서가 충분한 정보를 담지 못한 경우 외부 지식을 활용
      - 웹 검색 결과를 기존 문서에 추가 

   1. 답변 생성 (`generate_answer`):
      - 정제된 지식 조각을 사용하여 답변을 생성
      - 관련 정보가 없을 경우 적절한 메시지를 반환

- 논문: https://arxiv.org/pdf/2401.15884

----

# 1. 환경 설정

`(1) Env 환경변수`

In [ ]:
from dotenv import load_dotenv
load_dotenv()

`(2) 기본 라이브러리`

In [ ]:
import re
import os, json
from glob import glob

from textwrap import dedent
from pprint import pprint

import uuid

import warnings
warnings.filterwarnings("ignore")

# 2. 법률 문서를 로드하여 벡터저장소에 저장

In [ ]:
# pdf 파일 목록을 확인
pdf_files = glob(os.path.join('data', '*_law.pdf'))

pdf_files

###  2-1. 개인정보보호법

- PDf 문서를 가져와서 조항 별로 구분하여 정리

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

pdf_file = 'data/personal_info_law.pdf'

loader = PyPDFLoader(pdf_file) 
pages = loader.load()

len(pages)

In [ ]:
pages[0].metadata

In [ ]:
print(pages[0].page_content)

In [ ]:
def parse_law(law_text):
    # 서문 분리
    # '^'로 시작하여 '제1장' 또는 '제1조' 직전까지의 모든 텍스트를 탐색 
    preamble_pattern = r'^(.*?)(?=제1장|제1조)'
    preamble = re.search(preamble_pattern, law_text, re.DOTALL)
    if preamble:
        preamble = preamble.group(1).strip()
    
    # 장 분리 
    # '제X장' 형식의 제목과 그 뒤에 오는 모든 조항을 하나의 그룹화 
    chapter_pattern = r'(제\d+장\s+.+?)\n((?:제\d+조(?:의\d+)?(?:\(\w+\))?.*?)(?=제\d+장|부칙|$))'
    chapters = re.findall(chapter_pattern, law_text, re.DOTALL)
    
    # 부칙 분리
    # '부칙'으로 시작하는 모든 텍스트를 탐색 
    appendix_pattern = r'(부칙.*)'
    appendix = re.search(appendix_pattern, law_text, re.DOTALL)
    if appendix:
        appendix = appendix.group(1)
    
    # 파싱 결과를 저장할 딕셔너리 초기화
    parsed_law = {'서문': preamble, '장': {}, '부칙': appendix}
    
    # 각 장 내에서 조 분리
    for chapter_title, chapter_content in chapters:
        # 조 분리 패턴
        # 1. '제X조'로 시작 ('제X조의Y' 형식도 가능)
        # 2. 조 번호 뒤에 반드시 '(항목명)' 형식의 제목이 와야 함 
        # 3. 다음 조가 시작되기 전까지 또는 문서의 끝까지의 모든 내용을 포함
        article_pattern = r'(제\d+조(?:의\d+)?\s*\([^)]+\).*?)(?=제\d+조(?:의\d+)?\s*\([^)]+\)|$)'
        
        # 정규표현식을 이용해 모든 조항을 탐색 
        articles = re.findall(article_pattern, chapter_content, re.DOTALL)
        
        # 각 조항의 앞뒤 공백을 제거하고 결과 딕셔너리에 저장
        parsed_law['장'][chapter_title.strip()] = [article.strip() for article in articles]
    
    return parsed_law


# 각 페이지의 텍스트를 결합하여 재분리
text_for_delete = r"법제처\s+\d+\s+국가법령정보센터\n개인정보 보호법"

law_text = "\n".join([re.sub(text_for_delete, "", p.page_content).strip() for p in pages])

parsed_law = parse_law(law_text)

# 분할된 아이템 갯수 확인
print(len(parsed_law["장"]))

In [ ]:
parsed_law

- 랭체인 Document 객체에 메타데이터와 함께 정리

In [ ]:
from langchain_core.documents import Document

final_docs = []
for law in parsed_law['장'].keys():
    for article in parsed_law['장'][law]:

        # metadata 내용을 정리 
        metadata = {
                "source": pdf_file,
                "chapter": law,
                "name" : "개인정보 보호법"
                }

        # metadata 내용을 본문에 추가 
        content = f"<출처>\n다음 조항은 **{metadata['name']}** 본문의 **{metadata['chapter']}**에서 발췌한 내용입니다.\n</출처>\n\n<법률조항>\n{article}\n</법률조항>"

        final_docs.append(Document(page_content=content, metadata=metadata))
        
len(final_docs)


In [ ]:
print(final_docs[0].page_content)
print("-" * 100)
print(final_docs[0].metadata)

In [ ]:
print(final_docs[-1].page_content)
print("-" * 100)
print(final_docs[-1].metadata)

- 벡터저장소에 인덱싱

In [ ]:
# 각 문서의 텍스트 길이를 확인

text_lengths = [len(d.page_content) for d in final_docs]
print(min(text_lengths), max(text_lengths))

In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")

# Chroma 인덱스 생성
personal_db = Chroma.from_documents(
    documents=final_docs, 
    embedding=embeddings_model,   
    collection_name="personal_info_law",
    persist_directory="./chroma_db",
)

In [ ]:
print(personal_db._collection.count())  # 인덱스에 저장된 문서 수 출력

###  2-2. 근로기준법

- PDf 문서를 가져와서 조항 별로 구분하여 정리

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

pdf_file = 'data/labor_law.pdf'

loader = PyPDFLoader(pdf_file)
pages = loader.load()

len(pages)

In [ ]:
# 각 페이지의 텍스트를 결합하여 재분리
text_for_delete = r"법제처\s+\d+\s+국가법령정보센터\n근로기준법"

law_text = "\n".join([re.sub(text_for_delete, "", p.page_content).strip() for p in pages])

parsed_law = parse_law(law_text)

# 분할된 아이템 갯수 확인
print(len(parsed_law["장"]))

In [ ]:
parsed_law

- 랭체인 Document 객체에 메타데이터와 함께 정리

In [ ]:
from langchain_core.documents import Document

final_docs = []
for law in parsed_law['장'].keys():
    for article in parsed_law['장'][law]:

        # metadata 내용을 정리 
        metadata = {
                "source": pdf_file,
                "chapter": law,
                "name" : "근로기준법"
                }

        # metadata 내용을 본문에 추가 
        content = f"<출처>\n다음 조항은 **{metadata['name']}** 본문의 **{metadata['chapter']}**에서 발췌한 내용입니다.\n</출처>\n\n<법률조항>\n{article}\n</법률조항>"

        final_docs.append(Document(page_content=content, metadata=metadata))
        
len(final_docs)

In [ ]:
print(final_docs[0].page_content)
print("-" * 100)
print(final_docs[0].metadata)

In [ ]:
print(final_docs[1].page_content)
print("-" * 100)
print(final_docs[1].metadata)

- 벡터저장소에 인덱싱

In [ ]:
# 각 문서의 텍스트 길이를 확인

text_lengths = [len(d.page_content) for d in final_docs]
print(min(text_lengths), max(text_lengths))

In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")

# Chroma 인덱스 생성
labor_db = Chroma.from_documents(
    documents=final_docs, 
    embedding=embeddings_model,   
    collection_name="labor_law",
    persist_directory="./chroma_db",
)

In [ ]:
print(labor_db._collection.count())  # 인덱스에 저장된 문서 수 출력

###  2-3. 주택임대차보호법

- PDf 문서를 가져와서 조항 별로 구분하여 정리

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

pdf_file = 'data/housing_leasing_law.pdf'

loader = PyPDFLoader(pdf_file)
pages = loader.load()

len(pages)

In [ ]:
# 파싱 함수를 수정 (장이 없이 조문으로만 구성된 경우)
def parse_law_v2(law_text):
    # 서문 분리
    preamble_pattern = r'^(.*?)(?=제1장|제1조)'
    preamble = re.search(preamble_pattern, law_text, re.DOTALL)
    if preamble:
        preamble = preamble.group(1).strip()
    
    # 장 분리 
    chapter_pattern = r'(제\d+장\s+.+?)\n((?:제\d+조(?:의\d+)?(?:\(\w+\))?.*?)(?=제\d+장|부칙|$))'
    chapters = re.findall(chapter_pattern, law_text, re.DOTALL)
    
    # 부칙 분리
    appendix_pattern = r'(부칙.*)'
    appendix = re.search(appendix_pattern, law_text, re.DOTALL)
    if appendix:
        appendix = appendix.group(1)
    
    parsed_law = {'서문': preamble, '부칙': appendix}
    
    # 조 분리 패턴
    article_pattern = r'(제\d+조(?:의\d+)?\s*\([^)]+\).*?)(?=제\d+조(?:의\d+)?\s*\([^)]+\)|$)'
    
    if chapters:  # 장이 있는 경우
        parsed_law['장'] = {}
        for chapter_title, chapter_content in chapters:
            articles = re.findall(article_pattern, chapter_content, re.DOTALL)
            parsed_law['장'][chapter_title.strip()] = [article.strip() for article in articles]
    else:  # 장이 없는 경우
        # 서문과 부칙을 제외한 본문에서 조문 추출
        main_text = re.sub(preamble_pattern, '', law_text, flags=re.DOTALL)
        main_text = re.sub(appendix_pattern, '', main_text, flags=re.DOTALL)
        articles = re.findall(article_pattern, main_text, re.DOTALL)
        parsed_law['조문'] = [article.strip() for article in articles]
    
    return parsed_law

In [ ]:
# 각 페이지의 텍스트를 결합하여 재분리
text_for_delete = r"법제처\s+\d+\s+국가법령정보센터\n주택임대차보호법"

law_text = "\n".join([re.sub(text_for_delete, "", p.page_content).strip() for p in pages])

parsed_law = parse_law_v2(law_text)

# 분할된 아이템 갯수 확인
print(len(parsed_law["조문"]))

In [ ]:
parsed_law

- 랭체인 Document 객체에 메타데이터와 함께 정리

In [ ]:
from langchain_core.documents import Document

final_docs = []
for article in parsed_law['조문']:

    # metadata 내용을 정리 
    metadata = {
            "source": pdf_file,
            "name" : "주택임대차보호법"
            }

    # metadata 내용을 본문에 추가 
    content = f"<출처>\n다음 조항은 **{metadata['name']}**에서 발췌한 내용입니다.\n</출처>\n\n<법률조항>\n{article}\n</법률조항>"

    final_docs.append(Document(page_content=content, metadata=metadata))

        
len(final_docs)

In [ ]:
print(final_docs[0].page_content)
print("-" * 100)
print(final_docs[0].metadata)

In [ ]:
print(final_docs[1].page_content)
print("-" * 100)
print(final_docs[1].metadata)

- 벡터저장소에 인덱싱

In [ ]:
# 각 문서의 텍스트 길이를 확인

text_lengths = [len(d.page_content) for d in final_docs]
print(min(text_lengths), max(text_lengths))

In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")

# Chroma 인덱스 생성
housing_db = Chroma.from_documents(
    documents=final_docs, 
    embedding=embeddings_model,   
    collection_name="housing_law",
    persist_directory="./chroma_db",
)

In [ ]:
print(housing_db._collection.count())  # 인덱스에 저장된 문서 수 출력

# 3. 도구 호출

### 3-1. 법률 정보 검색 도구, 웹 검색 도구 정의

- `uv add tavily-python` 설치

In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document

from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain_community.retrievers import TavilySearchAPIRetriever
from langchain_core.tools import tool
from typing import List

# 문서 임베딩 모델
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")

# Re-rank 모델
rerank_model = HuggingFaceCrossEncoder(
    model_name="Alibaba-NLP/gte-multilingual-reranker-base",
    model_kwargs={
        "device": "cpu",  # CPU에서 실행
        "trust_remote_code": True,  # 모델이 외부 코드를 신뢰하도록 설정
        } 
    )
cross_reranker = CrossEncoderReranker(model=rerank_model, top_n=4)

# 개인정보보호법 검색 
personal_db = Chroma(
    embedding_function=embeddings_model,   
    collection_name="personal_info_law",
    persist_directory="./chroma_db",
)

personal_db_retriever = ContextualCompressionRetriever(
    base_compressor=cross_reranker, 
    base_retriever=personal_db.as_retriever(search_kwargs={"k":5}),
)

@tool
def personal_law_search(query: str) -> List[Document]:
    """개인정보보호법 법률 조항을 검색합니다."""
    docs = personal_db_retriever.invoke(query)

    if len(docs) > 0:
        return docs

    return [Document(page_content="**개인정보보호법 검색**에서 관련 정보를 찾을 수 없습니다.", metadata={"source": "data/personal_info_law.pdf", "name": "개인정보보호법"})]


# 근로기준법 검색 
labor_db = Chroma(
    embedding_function=embeddings_model,   
    collection_name="labor_law",
    persist_directory="./chroma_db",
)

labor_db_retriever = ContextualCompressionRetriever(
    base_compressor=cross_reranker, 
    base_retriever=labor_db.as_retriever(search_kwargs={"k":5}),
)


@tool
def labor_law_search(query: str) -> List[Document]:
    """근로기준법 법률 조항을 검색합니다."""
    docs = labor_db_retriever.invoke(query)

    if len(docs) > 0:
        return docs

    return [Document(page_content="**근로기준법 검색**에서 관련 정보를 찾을 수 없습니다.", metadata={"source": "data/labor_law.pdf", "name": "근로기준법"})]


# 주택임대차보호법 검색 
housing_db = Chroma(
    embedding_function=embeddings_model,   
    collection_name="housing_law",
    persist_directory="./chroma_db",
)

housing_db_retriever = ContextualCompressionRetriever(
    base_compressor=cross_reranker, 
    base_retriever=housing_db.as_retriever(search_kwargs={"k":5}),
)


@tool
def housing_law_search(query: str) -> List[Document]:
    """주택임대차보호법 법률 조항을 검색합니다."""
    docs = housing_db_retriever.invoke(query)

    if len(docs) > 0:
        return docs

    return [Document(page_content="**주택임대차보호법 검색**에서 관련 정보를 찾을 수 없습니다.", metadata={"source": "data/housing_leasing_law.pdf", "name": "주택임대차보호법"})]


# 웹 검색
web_retriever = ContextualCompressionRetriever(
    base_compressor=cross_reranker, 
    base_retriever=TavilySearchAPIRetriever(k=5),
)

@tool
def web_search(query: str) -> List[str]:
    """데이터베이스에 없는 정보 또는 최신 정보를 웹에서 검색합니다."""

    docs = web_retriever.invoke(query)

    formatted_docs = []
    for doc in docs:
        formatted_docs.append(
            Document(
                page_content= f'<Document href="{doc.metadata["source"]}"/>\n{doc.page_content}\n</Document>',
                metadata={"source": "web search", "url": doc.metadata["source"]}
            )
        )

    if len(formatted_docs) > 0:
        return formatted_docs

    return [Document(page_content="**웹 검색**에서 관련 정보를 찾을 수 없습니다.")]

In [ ]:
# 도구 목록을 정의 
tools = [personal_law_search, labor_law_search, housing_law_search, web_search]

### 3-2. LLM 모델

In [ ]:
from langchain_openai import ChatOpenAI

# 기본 LLM
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# LLM에 도구 바인딩하여 추가 
llm_with_tools = llm.bind_tools(tools)

In [ ]:
# 근로기준법과 관련된 질문을 하는 경우 -> 근로기준법 검색 도구를 호출  
query = "연차휴가 부여 기준에 대해서 설명해주세요."
ai_msg = llm_with_tools.invoke(query)

pprint(ai_msg)
print("-" * 100)

pprint(ai_msg.content)
print("-" * 100)

pprint(ai_msg.tool_calls)
print("-" * 100)

In [ ]:
# 도구들의 목적과 관련 없는 질문을 하는 경우 -> 도구 호출 없이 그대로 답변을 생성 
query = "안녕하세요?"
ai_msg = llm_with_tools.invoke(query)

pprint(ai_msg)
print("-" * 100)

pprint(ai_msg.content)
print("-" * 100)

pprint(ai_msg.tool_calls)
print("-" * 100)

In [ ]:
# 벡터 검색과 웹 검색이 모두 필요한 경우 
query = "연차휴가 부여 기준에 대해서 설명해주세요. 2023년 연차휴가 사용 비율은 어느 정도인가요?"
ai_msg = llm_with_tools.invoke(query)

pprint(ai_msg)
print("-" * 100)

pprint(ai_msg.content)
print("-" * 100)

pprint(ai_msg.tool_calls)
print("-" * 100)

# 4. Agent RAG 구현 

- 각 법률에 특화된 RAG 에이전트를 구현 
- 질문 라우팅을 통해서 각 에이전트를 도구 형태로 사용 
- 생성된 답변에 대한 피드백을 제공하는 에이전트 사용 
- 필요한 경우 사람의 피드백을 요청 (답변이 애매한 경우 - 재검색 여부 판단)


### 4-1. 각 법률에 특화된 RAG 에이전트를 구현 
- 검색된 문서의 관련성 등을 평가하여 질문 재작성 및 다시 검색 (Corrective RAG 적용)

`(1) 개인정보보호법 검색 에이전트`

In [ ]:
from pydantic import BaseModel, Field
from typing import List, TypedDict, Optional, Annotated, Literal
from langchain_core.documents import Document
import operator

class CorrectiveRagState(TypedDict):
    question: str                 # 사용자의 질문
    generation: str               # LLM 생성 답변
    documents: List[Document]     # 컨텍스트 문서 (검색된 문서)
    num_generations: int          # 질문 or 답변 생성 횟수 (무한 루프 방지에 활용)
    user_decision: Literal["rejected", "accepted"]  # 사용자 결정 (거부 or 수락)
    user_feedback: Optional[str]  # 사용자 피드백 (구체적인 피드백을 제공할 수 있는 경우)

class InformationStrip(BaseModel):
    """추출된 정보에 대한 내용과 출처, 관련성 점수"""
    content: str = Field(description="추출된 정보 내용")
    source: str = Field(description="정보의 출처(법률 조항 또는 URL 등). 예시: 환경법 제22조 3항 or 블로그 환경법 개정 (https://blog.com/page/123)")
    relevance_score: float = Field(description="관련성 점수 (0에서 1 사이)")
    faithfulness_score: float = Field(description="충실성 점수 (0에서 1 사이)")

class ExtractedInformation(BaseModel):
    strips: List[InformationStrip] = Field(description="추출된 정보 조각들")
    query_relevance: float = Field(description="질의에 대한 전반전인 답변 가능성 점수 (0에서 1 사이)")

class RefinedQuestion(BaseModel):
    """개선된 질문과 이유"""
    question_refined : str = Field(description="개선된 질문")
    reason : str = Field(description="이유")

# PersonalRagState - 병렬 처리를 지원
class PersonalRagState(CorrectiveRagState):
    rewritten_query: str   # 재작성한 질문 
    extracted_info: Annotated[List[InformationStrip], operator.add]  # 병렬 워커 작업용
    processed_info: List[InformationStrip]  # 후처리된 결과용 
    node_answer: Optional[str]

# 개별 문서 처리를 위한 워커 상태
class DocumentWorkerState(TypedDict):
    document: str  # 단일 문서
    question: str  # 질문
    # 워커에서도 동일한 타입으로 반환
    extracted_info: Annotated[List[InformationStrip], operator.add]

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.types import Send

def retrieve_documents(state: PersonalRagState) -> PersonalRagState:
    print("---개인정보보호법 문서 검색---")
    query = state.get("rewritten_query", state["question"])
    docs = personal_law_search.invoke(query)
    return {"documents": docs}

def create_evaluation_workers(state: PersonalRagState):
    """각 문서에 대해 평가 작업을 수행하는 병렬 워커를 생성하는 조건부 엣지 함수"""
    print(f"---개인정보보호법 관련 {len(state['documents'])}개 문서에 대한 평가 병렬 워커 생성---")

    # 각 문서에 대해 Send 객체를 생성하여 병렬 처리
    return [
        Send("evaluate_single_document", {
            "document": doc.page_content,
            "question": state["question"],
            "user_feedback": state.get("user_feedback", ""),  # 사용자 피드백이 있는 경우 전달
        })
        for doc in state["documents"]
    ]

def evaluate_single_document(state: DocumentWorkerState) -> PersonalRagState:
    """단일 문서를 처리하는 워커 함수"""
    print(f"---개인정보보호법 단일 문서 처리 중 (길이: {len(state['document'])}...)---")
    
    # 프롬프트 템플릿 정의
    extract_prompt = ChatPromptTemplate.from_messages([
        ("system", """당신은 개인정보보호법 전문가입니다. 주어진 문서에서 질문과 관련된 주요 사실과 정보를 3~5개 정도 추출하세요. 

        각 추출된 정보에 대해 다음 두 가지 측면을 0에서 1 사이의 점수로 평가하세요:
        1. 질문과의 관련성
        2. 답변의 충실성 (질문에 대한 완전하고 정확한 답변을 제공할 수 있는 정도)
        
        추출 형식:
        1. [추출된 정보]
        - 관련성 점수: [0-1 사이의 점수]
        - 충실성 점수: [0-1 사이의 점수]
        2. [추출된 정보]
        - 관련성 점수: [0-1 사이의 점수]
        - 충실성 점수: [0-1 사이의 점수]
        ...
        
        마지막으로, 추출된 정보를 종합하여 질문에 대한 전반적인 답변 가능성을 0에서 1 사이의 점수로 평가하세요."""),
        MessagesPlaceholder("user_feedback"),
        ("human", "<질문>\n{question}\n</질문>\n<문서 내용>\n{document_content}\n</문서 내용>")
    ])

    # LLM 호출 
    extract_llm = llm.with_structured_output(ExtractedInformation)
    
    try:
        extracted_data = extract_llm.invoke(extract_prompt.format(
            question=state["question"],
            document_content=state["document"],
            user_feedback=[("system", f"정보를 추출할 때 다음 사용자 피드백을 고려하세요:\n<피드백>{state['user_feedback']}</피드백>")] if "user_feedback" in state else []
        ))

        # 품질 필터링
        if extracted_data.query_relevance < 0.8:
            print(f"문서 관련성이 낮음: {extracted_data.query_relevance}")
            return {"extracted_info": []}

        # 고품질 정보만 필터링
        high_quality_strips = [
            strip for strip in extracted_data.strips
            if strip.relevance_score > 0.7 and strip.faithfulness_score > 0.7
        ]
        
        print(f"고품질 정보 {len(high_quality_strips)}개 추출됨")
        return {"extracted_info": high_quality_strips}
        
    except Exception as e:
        print(f"문서 처리 중 오류 발생: {e}")
        return {"extracted_info": []}

def aggregate_results(state: PersonalRagState) -> PersonalRagState:
    """병렬 처리된 결과를 집계하는 함수"""
    print(f"---개인정보보호법 관련 총 {len(state['extracted_info'])}개의 정보 조각 집계---")

    # 후처리가 필요한 경우에만 작업 추가 
    
    # 방법 1: 중복 제거
    unique_strips = []
    seen_content = set()
    for strip in state["extracted_info"]:
        if strip.content not in seen_content:
            unique_strips.append(strip)
            seen_content.add(strip.content)
    
    # 방법 2: 품질순 정렬
    sorted_strips = sorted(
        unique_strips, 
        key=lambda x: (x.relevance_score + x.faithfulness_score) / 2, 
        reverse=True
    )
    
    # 방법 3: 상위 N개만 선택
    top_strips = sorted_strips[:10]  # 상위 10개만
    
    return {
        "num_generations": state.get("num_generations", 0) + 1,
        "processed_info": top_strips  # 후처리된 결과
    }

In [ ]:
# 쿼리 재작성 함수
def rewrite_query(state: PersonalRagState) -> PersonalRagState:
    print("---개인정보보호법 쿼리 재작성---")

    rewrite_prompt = ChatPromptTemplate.from_messages([
        ("system", """당신은 개인정보보호법 전문가입니다. 주어진 원래 질문과 추출된 정보를 바탕으로, 더 관련성 있고 충실한 정보를 찾기 위해 검색 쿼리를 개선해주세요.

        다음 사항을 고려하여 검색 쿼리를 개선하세요:
        1. 원래 질문의 핵심 요소
        2. 추출된 정보의 관련성 점수
        3. 추출된 정보의 충실성 점수
        4. 부족한 정보나 더 자세히 알아야 할 부분

        개선된 검색 쿼리 작성 단계:
        1. 2-3개의 검색 쿼리를 제안하세요.
        2. 각 쿼리는 구체적이고 간결해야 합니다(5-10 단어 사이).
        3. 개인정보보호법과 관련된 전문 용어를 적절히 활용하세요.
        4. 각 쿼리 뒤에는 해당 쿼리를 제안한 이유를 간단히 설명하세요.

        출력 형식:
        1. [개선된 검색 쿼리 1]
        - 이유: [이 쿼리를 제안한 이유 설명]
        2. [개선된 검색 쿼리 2]
        - 이유: [이 쿼리를 제안한 이유 설명]
        3. [개선된 검색 쿼리 3]
        - 이유: [이 쿼리를 제안한 이유 설명]

        마지막으로, 제안된 쿼리 중 가장 효과적일 것 같은 쿼리를 선택하고 그 이유를 설명하세요."""),
        MessagesPlaceholder("user_feedback"),
        ("human", "<원래 질문>\n{question}\n</원래 질문>\n<추출된 정보>\n{processed_info}\n</추출된 정보>\n\n위 지침에 따라 개선된 검색 쿼리를 작성해주세요.")
    ])

    processed_info_str = "\n".join([strip.content for strip in state["processed_info"]])
    
    rewrite_llm = llm.with_structured_output(RefinedQuestion)

    response = rewrite_llm.invoke(rewrite_prompt.format(
        question=state["question"],
        processed_info=processed_info_str,
        user_feedback=[("system", f"쿼리를 재작성할 때 다음 사용자 피드백을 고려하세요:\n<피드백>{state.get('user_feedback', '')}</피드백>")] if "user_feedback" in state else []
    ))
    
    return {"rewritten_query": response.question_refined}

def generate_node_answer(state: PersonalRagState) -> PersonalRagState:
    print("---개인정보보호법 답변 생성---")

    answer_prompt = ChatPromptTemplate.from_messages([
        ("system", """당신은 개인정보보호법 전문가입니다. 주어진 질문과 추출된 정보를 바탕으로 답변을 생성해주세요. 
        답변은 마크다운 형식으로 작성하며, 각 정보의 출처를 명확히 표시해야 합니다. 
        답변 구조:
        1. 질문에 대한 직접적인 답변
        2. 관련 법률 조항 및 해석
        3. 추가 설명 또는 예시 (필요한 경우)
        4. 결론 및 요약
        각 섹션에서 사용된 정보의 출처를 괄호 안에 명시하세요. 예: (출처: 개인정보 보호법 제15조)"""),
        MessagesPlaceholder("user_feedback"),
        ("human", "<질문>\n{question}\n</질문>\n<추출된 정보>\n{processed_info}\n</추출된 정보>\n\n위 지침에 따라 최종 답변을 작성해주세요.")
    ])

    processed_info_str = "\n---\n".join([f"**내용**: {strip.content}\n**출처**: {strip.source}\n**관련성**: {strip.relevance_score}\n**충실성**: {strip.faithfulness_score}" for strip in state["processed_info"]])

    node_answer = llm.invoke(answer_prompt.format(
        question=state["question"],
            processed_info=processed_info_str,
        user_feedback=[("system", f"답변을 생성할 때 다음 사용자 피드백을 고려하세요:\n<피드백>{state.get('user_feedback', '')}</피드백>")] if "user_feedback" in state else []
    ))

    return {"node_answer": node_answer.content}

def should_continue(state: PersonalRagState) -> Literal["계속", "종료"]:
    if state["num_generations"] >= 2:
        return "종료"
    if len(state["processed_info"]) > 0:
        return "종료"
    return "계속"

In [ ]:
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display

# 그래프 생성
workflow = StateGraph(PersonalRagState)

workflow.add_node("retrieve", retrieve_documents)
workflow.add_node("evaluate_single_document", evaluate_single_document)
workflow.add_node("aggregate_results", aggregate_results)
workflow.add_node("rewrite_query", rewrite_query)
workflow.add_node("generate_answer", generate_node_answer)

workflow.add_edge(START, "retrieve")

# 조건부 엣지로 병렬 처리 설정
workflow.add_conditional_edges(
    "retrieve",  # 문서 검색 완료 후
    create_evaluation_workers,  # 조건부 엣지 함수
    ["evaluate_single_document"]  # 병렬 워커 노드
)

workflow.add_edge("evaluate_single_document", "aggregate_results")

# 조건부 엣지 추가
workflow.add_conditional_edges(
    "aggregate_results",
    should_continue,
    {
        "계속": "rewrite_query",
        "종료": "generate_answer"
    }
)
workflow.add_edge("rewrite_query", "retrieve")
workflow.add_edge("generate_answer", END)

# 그래프 컴파일
personal_law_agent = workflow.compile()

# 그래프 시각화
display(Image(personal_law_agent.get_graph().draw_mermaid_png()))

In [ ]:
inputs = {"question": "개인정보 처리에 대한 동의를 받을 때 주의해야 할 점은 무엇인가요?"}
for output in personal_law_agent.stream(inputs):
    for key, value in output.items():
        # 노드 출력
        pprint(f"Node '{key}':")
        pprint(f"Value: {value}", indent=2, width=80, depth=None)
    print("\n----------------------------------------------------------\n")

In [ ]:
# 마크다운 형식을 노트북에 표시
from IPython.display import Markdown
display(Markdown(value['node_answer']))

`(2) 근로기준법 RAG 에이전트`

In [ ]:
# 근로기준법
class LaborRagState(CorrectiveRagState):
    rewritten_query: str   # 재작성한 질문 
    extracted_info: Annotated[List[InformationStrip], operator.add]  # 병렬 워커들로부터 여러 정보 조각을 수집
    processed_info: List[InformationStrip]  # 후처리된 결과용 
    node_answer: Optional[str]

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from typing import Literal, List
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_transformers import EmbeddingsRedundantFilter
from langchain_core.documents import Document

def retrieve_documents(state: LaborRagState) -> LaborRagState:
    print("---근로기준법 문서 검색---")
    query = state.get("rewritten_query", state["question"])
    docs = labor_law_search.invoke(query)
    return {"documents": docs}


def create_evaluation_workers(state: LaborRagState):
    """각 문서에 대해 평가 작업을 수행하는 병렬 워커를 생성하는 조건부 엣지 함수"""
    print(f"---근로기준법 관련 {len(state['documents'])}개 문서에 대한 평가 병렬 워커 생성---")

    # 각 문서에 대해 Send 객체를 생성하여 병렬 처리
    return [
        Send("evaluate_single_document", {
            "document": doc.page_content,
            "question": state["question"]
        })
        for doc in state["documents"]
    ]


def evaluate_single_document(state: DocumentWorkerState) -> LaborRagState:
    """단일 문서를 처리하는 워커 함수"""
    print(f"---근로기준법 단일 문서 처리 중 (길이: {len(state['document'])}...)---")

    # 프롬프트 템플릿 정의
    extract_prompt = ChatPromptTemplate.from_messages([
        ("system", """당신은 근로기준법 전문가입니다. 주어진 문서에서 질문과 관련된 주요 사실과 정보를 3~5개 정도 추출하세요. 
        각 추출된 정보에 대해 다음 두 가지 측면을 0에서 1 사이의 점수로 평가하세요:
        1. 질문과의 관련성
        2. 답변의 충실성 (질문에 대한 완전하고 정확한 답변을 제공할 수 있는 정도)
        추출 형식:
        1. [추출된 정보]
        - 관련성 점수: [0-1 사이의 점수]
        - 충실성 점수: [0-1 사이의 점수]
        2. [추출된 정보]
        - 관련성 점수: [0-1 사이의 점수]
        - 충실성 점수: [0-1 사이의 점수]
        ...  
                    
        마지막으로, 추출된 정보를 종합하여 질문에 대한 전반적인 답변 가능성을 0에서 1 사이의 점수로 평가하세요."""),
        MessagesPlaceholder("user_feedback"),
        ("human", "<질문>\n{question}\n</질문>\n<문서 내용>\n{document_content}\n</문서 내용>")
    ])

    # LLM 호출
    extract_llm = llm.with_structured_output(ExtractedInformation)  
    try:    
        extracted_data = extract_llm.invoke(extract_prompt.format(
            question=state["question"],
            document_content=state["document"],
            user_feedback=[("system", f"정보를 추출할 때 다음 사용자 피드백을 고려하세요:\n<피드백>{state.get('user_feedback', '')}</피드백>")] if "user_feedback" in state else []
        ))

        # 품질 필터링
        if extracted_data.query_relevance < 0.8:
            print(f"문서 관련성이 낮음: {extracted_data.query_relevance}")
            return {"extracted_info": []}

        # 고품질 정보만 필터링
        high_quality_strips = [
            strip for strip in extracted_data.strips
            if strip.relevance_score > 0.7 and strip.faithfulness_score > 0.7
        ]
        
        print(f"고품질 정보 {len(high_quality_strips)}개 추출됨")
        return {"extracted_info": high_quality_strips}
    
    except Exception as e:
        print(f"문서 처리 중 오류 발생: {e}")
        return {"extracted_info": []}
    


# 임베딩 모델
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# LangChain의 EmbeddingsRedundantFilter 초기화
redundant_filter = EmbeddingsRedundantFilter(
    embeddings=embeddings,
    similarity_threshold=0.85,  # 85% 이상 유사하면 중복으로 판단
)

def remove_semantic_duplicates(info_list: List[InformationStrip], 
                              similarity_threshold: float = 0.85) -> List[InformationStrip]:
    """LangChain EmbeddingsRedundantFilter를 사용한 의미적 중복 제거"""
    if not info_list:
        return []
    
    if len(info_list) == 1:
        return info_list
    
    try:
        # InformationStrip을 Document로 변환
        documents = []
        for i, info in enumerate(info_list):
            doc = Document(
                page_content=info.content,
                metadata={
                    "index": i,
                    "source": info.source,
                    "relevance_score": info.relevance_score,
                    "faithfulness_score": info.faithfulness_score
                }
            )
            documents.append(doc)
        
        # 임계값이 기본값과 다르면 새로운 필터 생성
        if similarity_threshold != 0.85:
            filter_instance = EmbeddingsRedundantFilter(
                embeddings=embeddings,
                similarity_threshold=similarity_threshold,
            )
        else:
            filter_instance = redundant_filter
        
        # 중복 제거 수행
        unique_documents = filter_instance.transform_documents(documents)
        
        # Document를 다시 InformationStrip으로 변환
        unique_info = []
        for doc in unique_documents:
            original_index = doc.metadata["index"]
            unique_info.append(info_list[original_index])
        
        # 품질 점수 순으로 정렬
        unique_info.sort(
            key=lambda x: (x.relevance_score + x.faithfulness_score) / 2, 
            reverse=True
        )
        
        print(f"EmbeddingsRedundantFilter 중복 제거: {len(info_list)} → {len(unique_info)}개")
        return unique_info
        
    except Exception as e:
        print(f"EmbeddingsRedundantFilter 처리 중 오류: {e}")
        # 폴백: 기존 문자열 기반 중복 제거
        return remove_simple_duplicates(info_list)



def aggregate_results(state: LaborRagState) -> LaborRagState:
    """병렬 처리된 결과를 집계하는 함수"""
    print(f"---근로기준법 관련 총 {len(state['extracted_info'])}개의 정보 조각 집계---")
    
    if not state["extracted_info"]:
        return {
            "num_generations": state.get("num_generations", 0) + 1,
            "processed_info": []
        }
    
    # 1. LangChain EmbeddingsRedundantFilter를 사용한 의미적 중복 제거
    unique_info = remove_semantic_duplicates(
        state["extracted_info"], 
        similarity_threshold=0.85  # 85% 이상 유사하면 중복으로 판단
    )
    
    # 2. 품질 점수에 따른 정렬
    sorted_info = sorted(
        unique_info, 
        key=lambda x: (x.relevance_score + x.faithfulness_score) / 2, 
        reverse=True
    )
    
    # 3. 상위 N개만 선택 
    top_info = sorted_info[:10]  # 상위 10개만 유지
    
    print(f"최종 결과: {len(state['extracted_info'])} → {len(unique_info)}  → {len(top_info)}개")
    
    # 중요: extracted_info 키를 반환하지 않음 (reducer 중복 방지)
    return {
        "num_generations": state.get("num_generations", 0) + 1,
        "processed_info": top_info  # 후처리된 결과
    }


def rewrite_query(state: LaborRagState) -> LaborRagState:
    print("---근로기준법 쿼리 재작성---")

    rewrite_prompt = ChatPromptTemplate.from_messages([
        ("system", """당신은 근로기준법 전문가입니다. 주어진 원래 질문과 추출된 정보를 바탕으로, 더 관련성 있고 충실한 정보를 찾기 위해 검색 쿼리를 개선해주세요.

        다음 사항을 고려하여 검색 쿼리를 개선하세요:
        1. 원래 질문의 핵심 요소
        2. 추출된 정보의 관련성 점수
        3. 추출된 정보의 충실성 점수
        4. 부족한 정보나 더 자세히 알아야 할 부분

        개선된 검색 쿼리 작성 단계:
        1. 2-3개의 검색 쿼리를 제안하세요.
        2. 각 쿼리는 구체적이고 간결해야 합니다(5-10 단어 사이).
        3. 근로기준법과 관련된 전문 용어를 적절히 활용하세요.
        4. 각 쿼리 뒤에는 해당 쿼리를 제안한 이유를 간단히 설명하세요.

        출력 형식:
        1. [개선된 검색 쿼리 1]
        - 이유: [이 쿼리를 제안한 이유 설명]
        2. [개선된 검색 쿼리 2]
        - 이유: [이 쿼리를 제안한 이유 설명]
        3. [개선된 검색 쿼리 3]
        - 이유: [이 쿼리를 제안한 이유 설명]

        마지막으로, 제안된 쿼리 중 가장 효과적일 것 같은 쿼리를 선택하고 그 이유를 설명하세요."""),
        MessagesPlaceholder("user_feedback"),
        ("human", "<원래 질문>\n{question}\n</원래 질문>\n\n<추출된 정보>\n{processed_info}\n</추출된 정보>\n\n위 지침에 따라 개선된 검색 쿼리를 작성해주세요.")
    ])

    processed_info_str = "\n".join([strip.content for strip in state["processed_info"]])
    
    rewrite_llm = llm.with_structured_output(RefinedQuestion)

    response = rewrite_llm.invoke(rewrite_prompt.format(
        question=state["question"],
        processed_info=processed_info_str,
        user_feedback=[("system", f"쿼리를 재작성할 때 다음 사용자 피드백을 고려하세요:\n<피드백>{state.get('user_feedback', '')}</피드백>")] if "user_feedback" in state else []
    ))
    
    return {"rewritten_query": response.question_refined}


def generate_node_answer(state: LaborRagState) -> LaborRagState:
    print("---근로기준법 답변 생성---")

    answer_prompt = ChatPromptTemplate.from_messages([
        ("system", """당신은 근로기준법 전문가입니다. 주어진 질문과 추출된 정보를 바탕으로 답변을 생성해주세요. 
        답변은 마크다운 형식으로 작성하며, 각 정보의 출처를 명확히 표시해야 합니다. 
        답변 구조:
        1. 질문에 대한 직접적인 답변
        2. 관련 법률 조항 및 해석
        3. 추가 설명 또는 예시 (필요한 경우)
        4. 결론 및 요약
        각 섹션에서 사용된 정보의 출처를 괄호 안에 명시하세요. 예: (출처: 근로기준법 제15조)"""),
        MessagesPlaceholder("user_feedback"),
        ("human", "<질문>\n{question}\n</질문>\n\n<추출된 정보>\n{processed_info}\n</추출된 정보>\n\n위 지침에 따라 최종 답변을 작성해주세요.")
    ])

    processed_info_str = "\n---\n".join([f"**내용**: {strip.content}\n**출처**: {strip.source}\n**관련성**: {strip.relevance_score}\n**충실성**: {strip.faithfulness_score}" for strip in state["processed_info"]])

    node_answer = llm.invoke(answer_prompt.format(
        question=state["question"],
        processed_info=processed_info_str,
        user_feedback=[("system", f"답변을 생성할 때 다음 사용자 피드백을 고려하세요:\n<피드백>{state.get('user_feedback', '')}</피드백>")] if "user_feedback" in state else []
    ))

    return {"node_answer": node_answer.content}

def should_continue(state: LaborRagState) -> Literal["계속", "종료"]:
    if state["num_generations"] >= 2:
        return "종료"
    if len(state["processed_info"]) > 0:
        return "종료"
    return "계속"

In [ ]:
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display

# 그래프 생성
workflow = StateGraph(LaborRagState)

# 노드 추가
workflow.add_node("retrieve", retrieve_documents)
workflow.add_node("evaluate_single_document", evaluate_single_document)
workflow.add_node("aggregate_results", aggregate_results)
workflow.add_node("rewrite_query", rewrite_query)
workflow.add_node("generate_answer", generate_node_answer)  

workflow.add_edge(START, "retrieve")

# 조건부 엣지로 병렬 처리 설정
workflow.add_conditional_edges(
    "retrieve",  # 문서 검색 완료 후
    create_evaluation_workers,  # 조건부 엣지 함수
    ["evaluate_single_document"]  # 병렬 워커 노드
)

workflow.add_edge("evaluate_single_document", "aggregate_results")
workflow.add_conditional_edges(
    "aggregate_results",
    should_continue,
    {
        "계속": "rewrite_query",
        "종료": "generate_answer"
    }
)
workflow.add_edge("rewrite_query", "retrieve")
workflow.add_edge("generate_answer", END)

# 그래프 컴파일
labor_law_agent = workflow.compile()

# 그래프 시각화
display(Image(labor_law_agent.get_graph().draw_mermaid_png()))

In [ ]:
inputs = {"question": "근로계약 체결할 때 주의해야 할 점은 무엇인가요?"}
for output in labor_law_agent.stream(inputs):
    for key, value in output.items():
        # 노드 출력
        pprint(f"Node '{key}':")
        pprint(f"Value: {value}", indent=2, width=80, depth=None)
    print("\n----------------------------------------------------------\n")

In [ ]:
# 마크다운 형식으로 출력
from IPython.display import Markdown
display(Markdown(value['node_answer']))

`(3) 주택임대차보호법 RAG 에이전트`

In [ ]:
# 주택임대차보호법
class HousingRagState(CorrectiveRagState):
    rewritten_query: str   # 재작성한 질문 
    extracted_info: Annotated[List[InformationStrip], operator.add]  # 병렬 워커들로부터 여러 정보 조각을 수집
    processed_info: List[InformationStrip]  # 후처리된 결과용 
    node_answer: Optional[str]

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langgraph.types import Send

def retrieve_documents(state: HousingRagState) -> HousingRagState:
    print("---주택임대차보호법 문서 검색---")
    query = state.get("rewritten_query", state["question"])
    docs = housing_law_search.invoke(query)
    return {"documents": docs}

def create_evaluation_workers(state: HousingRagState):
    """각 문서에 대해 평가 작업을 수행하는 병렬 워커를 생성하는 조건부 엣지 함수"""
    print(f"---주택임대차보호법 관련 {len(state['documents'])}개 문서에 대한 평가 병렬 워커 생성---")

    # 각 문서에 대해 Send 객체를 생성하여 병렬 처리
    return [
        Send("evaluate_single_document", {
            "document": doc.page_content,
            "question": state["question"],
            "user_feedback": state.get("user_feedback", ""),  # 사용자 피드백이 있는 경우 전달
        })
        for doc in state["documents"]
    ]

def evaluate_single_document(state: DocumentWorkerState) -> HousingRagState:
    """단일 문서를 처리하는 워커 함수"""
    print(f"---주택임대차보호법 관련 단일 문서 처리 중 (길이: {len(state['document'])}...)---")
    
    # 프롬프트 템플릿 정의
    extract_prompt = ChatPromptTemplate.from_messages([
        ("system", """당신은 주택임대차보호법 전문가입니다. 주어진 문서에서 질문과 관련된 주요 사실과 정보를 3~5개 정도 추출하세요. 
        각 추출된 정보에 대해 다음 두 가지 측면을 0에서 1 사이의 점수로 평가하세요:
        1. 질문과의 관련성
        2. 답변의 충실성 (질문에 대한 완전하고 정확한 답변을 제공할 수 있는 정도)
        
        추출 형식:
        1. [추출된 정보]
        - 관련성 점수: [0-1 사이의 점수]
        - 충실성 점수: [0-1 사이의 점수]
        2. [추출된 정보]
        - 관련성 점수: [0-1 사이의 점수]
        - 충실성 점수: [0-1 사이의 점수]
        ...
        
        마지막으로, 추출된 정보를 종합하여 질문에 대한 전반적인 답변 가능성을 0에서 1 사이의 점수로 평가하세요."""),
        MessagesPlaceholder("user_feedback"),
        ("human", "<질문>\n{question}\n</질문>\n\n<문서 내용>\n{document_content}\n</문서 내용>")
    ])

    # LLM 호출 
    extract_llm = llm.with_structured_output(ExtractedInformation)
    
    try:
        extracted_data = extract_llm.invoke(extract_prompt.format(
            question=state["question"],
            document_content=state["document"],
            user_feedback=[("system", f"정보를 추출할 때 다음 사용자 피드백을 고려하세요:\n<피드백>{state.get('user_feedback', '')}</피드백>")] if "user_feedback" in state else []
        ))

        # 품질 필터링
        if extracted_data.query_relevance < 0.8:
            print(f"문서 관련성이 낮음: {extracted_data.query_relevance}")
            return {"extracted_info": []}

        # 고품질 정보만 필터링
        high_quality_strips = [
            strip for strip in extracted_data.strips
            if strip.relevance_score > 0.7 and strip.faithfulness_score > 0.7
        ]
        
        print(f"고품질 정보 {len(high_quality_strips)}개 추출됨")
        return {"extracted_info": high_quality_strips}
        
    except Exception as e:
        print(f"문서 처리 중 오류 발생: {e}")
        return {"extracted_info": []}

def aggregate_results(state: HousingRagState) -> HousingRagState:
    """병렬 처리된 결과를 집계하는 함수"""
    print(f"---주택임대차보호법 관련 총 {len(state['extracted_info'])}개의 정보 조각 집계---")
    
    if not state["extracted_info"]:
        return {
            "num_generations": state.get("num_generations", 0) + 1,
            "processed_info": []
        }
    
    # 1. LangChain EmbeddingsRedundantFilter를 사용한 의미적 중복 제거
    unique_info = remove_semantic_duplicates(
        state["extracted_info"], 
        similarity_threshold=0.85  # 85% 이상 유사하면 중복으로 판단
    )
    
    # 2. 품질 점수에 따른 정렬
    sorted_info = sorted(
        unique_info, 
        key=lambda x: (x.relevance_score + x.faithfulness_score) / 2, 
        reverse=True
    )
    
    # 3. 상위 N개만 선택 
    top_info = sorted_info[:10]  # 상위 10개만 유지
    
    print(f"최종 결과: {len(state['extracted_info'])} → {len(unique_info)}  → {len(top_info)}개")
    
    # 중요: extracted_info 키를 반환하지 않음 (reducer 중복 방지)
    return {
        "num_generations": state.get("num_generations", 0) + 1,
        "processed_info": top_info  # 후처리된 결과
    }

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from typing import Literal

def rewrite_query(state: HousingRagState) -> HousingRagState:
    print("---주택임대차보호법 쿼리 재작성---")

    rewrite_prompt = ChatPromptTemplate.from_messages([
        ("system", """당신은 주택임대차보호법 전문가입니다. 주어진 원래 질문과 추출된 정보를 바탕으로, 더 관련성 있고 충실한 정보를 찾기 위해 검색 쿼리를 개선해주세요.

        다음 사항을 고려하여 검색 쿼리를 개선하세요:
        1. 원래 질문의 핵심 요소
        2. 추출된 정보의 관련성 점수
        3. 추출된 정보의 충실성 점수
        4. 부족한 정보나 더 자세히 알아야 할 부분

        개선된 검색 쿼리 작성 단계:
        1. 2-3개의 검색 쿼리를 제안하세요.
        2. 각 쿼리는 구체적이고 간결해야 합니다(5-10 단어 사이).
        3. 주택임대차보호법과 관련된 전문 용어를 적절히 활용하세요.
        4. 각 쿼리 뒤에는 해당 쿼리를 제안한 이유를 간단히 설명하세요.

        출력 형식:
        1. [개선된 검색 쿼리 1]
        - 이유: [이 쿼리를 제안한 이유 설명]
        2. [개선된 검색 쿼리 2]
        - 이유: [이 쿼리를 제안한 이유 설명]
        3. [개선된 검색 쿼리 3]
        - 이유: [이 쿼리를 제안한 이유 설명]

        마지막으로, 제안된 쿼리 중 가장 효과적일 것 같은 쿼리를 선택하고 그 이유를 설명하세요."""),
        MessagesPlaceholder("user_feedback"),
        ("human", "<원래 질문>\n{question}\n</원래 질문>\n\n<추출된 정보>\n{processed_info}\n</추출된 정보>\n\n위 지침에 따라 개선된 검색 쿼리를 작성해주세요.")
    ])

    processed_info_str = "\n".join([strip.content for strip in state["processed_info"]])
    
    rewrite_llm = llm.with_structured_output(RefinedQuestion)

    response = rewrite_llm.invoke(rewrite_prompt.format(
        question=state["question"],
        processed_info=processed_info_str,
        user_feedback=[("system", f"쿼리를 재작성할 때 다음 사용자 피드백을 고려하세요:\n<피드백>{state.get('user_feedback', '')}</피드백>")] if "user_feedback" in state else []
    ))

    return {"rewritten_query": response.question_refined}

def generate_node_answer(state: HousingRagState) -> HousingRagState:
    print("---주택임대차보호법 답변 생성---")

    answer_prompt = ChatPromptTemplate.from_messages([
        ("system", """당신은 주택임대차보호법 전문가입니다. 주어진 질문과 추출된 정보를 바탕으로 답변을 생성해주세요. 
        답변은 마크다운 형식으로 작성하며, 각 정보의 출처를 명확히 표시해야 합니다. 
        답변 구조:
        1. 질문에 대한 직접적인 답변
        2. 관련 법률 조항 및 해석
        3. 추가 설명 또는 예시 (필요한 경우)
        4. 결론 및 요약
        각 섹션에서 사용된 정보의 출처를 괄호 안에 명시하세요. 예: (출처: 주택임대차보호법 제15조)"""),
        MessagesPlaceholder("user_feedback"),
        ("human", "<질문>\n{question}\n</질문>\n\n<추출된 정보>\n{processed_info}\n</추출된 정보>\n\n위 지침에 따라 최종 답변을 작성해주세요.")
    ])

    processed_info_str = "\n---\n".join([f"**내용**: {strip.content}\n**출처**: {strip.source}\n**관련성**: {strip.relevance_score}\n**충실성**: {strip.faithfulness_score}" for strip in state["processed_info"]])

    node_answer = llm.invoke(answer_prompt.format(
        question=state["question"],
        processed_info=processed_info_str,
        user_feedback=[("system", f"답변을 생성할 때 다음 사용자 피드백을 고려하세요:\n<피드백>{state.get('user_feedback', '')}</피드백>")] if "user_feedback" in state else []
    ))

    return {"node_answer": node_answer.content}

def should_continue(state: HousingRagState) -> Literal["계속", "종료"]:
    if state["num_generations"] >= 2:
        return "종료"
    if len(state["processed_info"]) > 0:
        return "종료"
    return "계속"

In [ ]:
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display

# 그래프 생성
workflow = StateGraph(HousingRagState)

# 노드 추가
workflow.add_node("retrieve", retrieve_documents)
workflow.add_node("evaluate_single_document", evaluate_single_document)
workflow.add_node("rewrite_query", rewrite_query)
workflow.add_node("generate_answer", generate_node_answer)
workflow.add_node("aggregate_results", aggregate_results)

workflow.add_edge(START, "retrieve")
# 조건부 엣지로 병렬 처리 설정
workflow.add_conditional_edges(
    "retrieve",  # 문서 검색 완료 후
    create_evaluation_workers,  # 조건부 엣지 함수
    ["evaluate_single_document"]  # 병렬 워커 노드
)
workflow.add_edge("evaluate_single_document", "aggregate_results")
workflow.add_conditional_edges(
    "aggregate_results",
    should_continue,
    {
        "계속": "rewrite_query",
        "종료": "generate_answer"
    }
)

workflow.add_edge("rewrite_query", "retrieve")
workflow.add_edge("generate_answer", END)

# 그래프 컴파일
housing_law_agent = workflow.compile()

# 그래프 시각화
display(Image(housing_law_agent.get_graph().draw_mermaid_png()))

In [ ]:
inputs = {"question": "대리인과 아파트 임대차 계약을 체결할 때 주의해야 할 점은 무엇인가요?"}
for output in housing_law_agent.stream(inputs):
    for key, value in output.items():
        # 노드 출력
        pprint(f"Node '{key}':")
        pprint(f"Value: {value}", indent=2, width=80, depth=None)
    print("\n----------------------------------------------------------\n")

In [ ]:
# 마크다운 형식으로 출력
from IPython.display import Markdown
display(Markdown(value['node_answer']))

`(4) 웹 검색 기반 RAG 에이전트`

In [ ]:
# 웹 검색 도구 
class SearchRagState(CorrectiveRagState):
    rewritten_query: str   # 재작성한 질문 
    extracted_info: Annotated[List[InformationStrip], operator.add]  # 병렬 워커들로부터 여러 정보 조각을 수집
    processed_info: List[InformationStrip]  # 후처리된 결과용 
    node_answer: Optional[str]

In [ ]:
def retrieve_documents(state: SearchRagState) -> dict:
    """웹 문서 검색 함수"""
    print("---웹 문서 검색---")
    query = state.get("rewritten_query", state["question"])
    docs = web_search.invoke(query)
    return {"documents": docs}

def create_evaluation_workers(state: SearchRagState):
    """각 웹 문서에 대해 평가 작업을 수행하는 병렬 워커를 생성하는 조건부 엣지 함수"""
    print(f"---{len(state['documents'])}개 웹 문서에 대한 평가 병렬 워커 생성---")

    # 각 문서에 대해 Send 객체를 생성하여 병렬 처리
    return [
        Send("evaluate_single_document", {
            "document": doc.page_content,
            "question": state["question"],
            "user_feedback": state.get("user_feedback", ""),  # 사용자 피드백이 있는 경우 전달
        })
        for doc in state["documents"]
    ]

def evaluate_single_document(state: DocumentWorkerState) -> dict:
    """단일 웹 문서를 처리하는 워커 함수"""
    print(f"---단일 웹 문서 처리 중 (길이: {len(state['document'][:100])}...)---")
    
    # 웹 검색에 특화된 프롬프트 템플릿
    extract_prompt = ChatPromptTemplate.from_messages([
        ("system", """당신은 인터넷 정보 검색 전문가입니다. 주어진 웹 문서에서 질문과 관련된 주요 사실과 정보를 3~5개 정도 추출하세요. 
        
        웹 정보 추출 시 주의사항:
        - 신뢰할 수 있는 정보인지 판단하세요
        - 날짜가 명시된 정보는 시기를 포함하여 추출하세요
        - 통계나 수치 정보는 정확히 추출하세요
        - 출처 URL이나 사이트 정보를 포함하세요
        
        각 추출된 정보에 대해 다음 두 가지 측면을 0에서 1 사이의 점수로 평가하세요:
        1. 질문과의 관련성
        2. 답변의 충실성 (질문에 대한 완전하고 정확한 답변을 제공할 수 있는 정도)
        
        추출 형식:
        1. [추출된 정보]
        - 관련성 점수: [0-1 사이의 점수]
        - 충실성 점수: [0-1 사이의 점수]
        2. [추출된 정보]
        - 관련성 점수: [0-1 사이의 점수]
        - 충실성 점수: [0-1 사이의 점수]
        ...
        
        마지막으로, 추출된 정보를 종합하여 질문에 대한 전반적인 답변 가능성을 0에서 1 사이의 점수로 평가하세요."""),
        MessagesPlaceholder("user_feedback"),
        ("human", "<질문>\n{question}\n</질문>\n<웹 문서 내용>\n{document_content}\n</웹 문서 내용>")
    ])

    extract_llm = llm.with_structured_output(ExtractedInformation)
    
    try:
        extracted_data = extract_llm.invoke(extract_prompt.format(
            question=state["question"],
            document_content=state["document"],
            user_feedback=[("system", f"정보를 추출할 때 다음 사용자 피드백을 고려하세요:\n<피드백>{state.get('user_feedback', '')}</피드백>")] if "user_feedback" in state else []
        ))

        # 웹 정보의 품질 필터링 (조금 더 엄격)
        if extracted_data.query_relevance < 0.7:  # 웹 정보는 노이즈가 많으므로
            print(f"웹 문서 관련성이 낮음: {extracted_data.query_relevance}")
            return {"extracted_info": []}

        # 고품질 정보만 필터링
        high_quality_strips = [
            strip for strip in extracted_data.strips
            if strip.relevance_score > 0.6 and strip.faithfulness_score > 0.6  # 웹 정보 특성상 조금 낮춤
        ]
        
        print(f"고품질 웹 정보 {len(high_quality_strips)}개 추출됨")
        return {"extracted_info": high_quality_strips}
        
    except Exception as e:
        print(f"웹 문서 처리 중 오류 발생: {e}")
        return {"extracted_info": []}

def aggregate_results(state: SearchRagState) -> dict:
    """병렬 처리된 웹 검색 결과를 집계하는 함수"""
    print(f"---총 {len(state['extracted_info'])}개의 웹 정보 조각 집계---")
    
    if not state["extracted_info"]:
        return {
            "num_generations": state.get("num_generations", 0) + 1,
            "processed_info": []
        }
    
    # 1. LangChain EmbeddingsRedundantFilter를 사용한 의미적 중복 제거
    unique_info = remove_semantic_duplicates(
        state["extracted_info"], 
        similarity_threshold=0.85  # 85% 이상 유사하면 중복으로 판단
    )
    
    # 2. 품질 점수에 따른 정렬
    sorted_info = sorted(
        unique_info, 
        key=lambda x: (x.relevance_score + x.faithfulness_score) / 2, 
        reverse=True
    )
    
    # 3. 상위 N개만 선택 
    top_info = sorted_info[:10]  # 상위 10개만 유지
    
    print(f"최종 결과: {len(state['extracted_info'])} → {len(unique_info)}  → {len(top_info)}개")
    
    # 중요: extracted_info 키를 반환하지 않음 (reducer 중복 방지)
    return {
        "num_generations": state.get("num_generations", 0) + 1,
        "processed_info": top_info  # 후처리된 결과
    }

def rewrite_query(state: SearchRagState) -> dict:
    """웹 검색 쿼리 재작성 함수"""
    print("---웹 검색 쿼리 재작성---")

    rewrite_prompt = ChatPromptTemplate.from_messages([
        ("system", """당신은 인터넷 정보 검색 전문가입니다. 주어진 원래 질문과 추출된 웹 정보를 바탕으로, 더 관련성 있고 충실한 정보를 찾기 위해 검색 쿼리를 개선해주세요.

        웹 검색 쿼리 개선 가이드라인:
        1. 원래 질문의 핵심 키워드 유지
        2. 추출된 정보에서 부족한 부분 파악
        3. 시간적 맥락이 중요한 경우 연도 추가
        4. 구체적인 용어나 전문 용어 활용
        5. 검색 엔진 최적화를 고려한 키워드 조합

        다음 사항을 고려하여 검색 쿼리를 개선하세요:
        1. 원래 질문의 핵심 요소
        2. 추출된 정보의 관련성 점수
        3. 추출된 정보의 충실성 점수
        4. 부족한 정보나 더 자세히 알아야 할 부분

        개선된 검색 쿼리 작성 단계:
        1. 2-3개의 검색 쿼리를 제안하세요.
        2. 각 쿼리는 구체적이고 간결해야 합니다(5-15 단어 사이).
        3. 질문과 관련된 전문 용어를 적절히 활용하세요.
        4. 각 쿼리 뒤에는 해당 쿼리를 제안한 이유를 간단히 설명하세요.

        출력 형식:
        1. [개선된 검색 쿼리 1]
        - 이유: [이 쿼리를 제안한 이유 설명]
        2. [개선된 검색 쿼리 2]
        - 이유: [이 쿼리를 제안한 이유 설명]
        3. [개선된 검색 쿼리 3]
        - 이유: [이 쿼리를 제안한 이유 설명]

        마지막으로, 제안된 쿼리 중 가장 효과적일 것 같은 쿼리를 선택하고 그 이유를 설명하세요."""),
        MessagesPlaceholder("user_feedback"),
        ("human", "<원래 질문>\n{question}\n</원래 질문>\n<웹 문서 내용>\n{processed_info}\n<</웹 문서 내용>\n\n위 지침에 따라 개선된 웹 검색 쿼리를 작성해주세요.")
    ])

    processed_info_str = "\n".join([
        f"- {strip.content} (출처: {strip.source})" 
        for strip in state["processed_info"]
    ])
    
    rewrite_llm = llm.with_structured_output(RefinedQuestion)

    response = rewrite_llm.invoke(rewrite_prompt.format(
        question=state["question"],
        processed_info=processed_info_str,
        user_feedback=[("system", f"쿼리를 재작성할 때 다음 사용자 피드백을 고려하세요:\n<피드백>{state.get('user_feedback', '')}</피드백>")] if "user_feedback" in state else []
    ))
    
    return {"rewritten_query": response.question_refined}

def generate_node_answer(state: SearchRagState) -> dict:
    """웹 검색 결과 기반 답변 생성 함수"""
    print("---웹 검색 기반 답변 생성---")

    answer_prompt = ChatPromptTemplate.from_messages([
        ("system", """당신은 인터넷 정보 검색 전문가입니다. 주어진 질문과 웹에서 추출된 정보를 바탕으로 신뢰할 수 있는 답변을 생성해주세요. 
        
        웹 정보 기반 답변 작성 가이드라인:
        - 여러 출처의 정보를 종합하여 균형잡힌 답변 제공
        - 정보의 신뢰도와 출처를 명확히 표시
        - 상충되는 정보가 있다면 명시
        - 정보의 시점이나 업데이트 날짜 고려
        - 추가 검증이 필요한 정보는 명시
        
        답변은 마크다운 형식으로 작성하며, 각 정보의 출처를 명확히 표시해야 합니다. 
        답변 구조:
        1. 질문에 대한 직접적인 답변 요약
        2. 상세 설명 (웹 정보 기반)
        3. 관련 출처 및 링크
        4. 주의사항 또는 추가 고려사항 (있는 경우)
        5. 결론 및 요약
        
        각 섹션에서 사용된 정보의 출처를 괄호 안에 명시하세요. 
        예: (출처: 네이버 뉴스, 2024.01.15) 또는 (출처: 위키피디아)"""),
        MessagesPlaceholder("user_feedback"),
        ("human", "<질문>\n{question}\n</질문>\n<웹에서 추출된 정보>\n{processed_info}\n</웹에서 추출된 정보>\n\n위 지침에 따라 신뢰할 수 있는 최종 답변을 작성해주세요.")
    ])

    # 웹 정보를 더 상세하게 포맷팅
    processed_info_str = "\n---\n".join([
        f"**내용**: {strip.content}\n"
        f"**출처**: {strip.source}\n"
        f"**신뢰도**: 관련성 {strip.relevance_score:.2f}, 충실성 {strip.faithfulness_score:.2f}"
        for strip in state["processed_info"]
    ])

    node_answer = llm.invoke(answer_prompt.format(
        question=state["question"],
        processed_info=processed_info_str,
        user_feedback=[("system", f"답변을 생성할 때 다음 사용자 피드백을 고려하세요:\n<피드백>{state.get('user_feedback', '')}</피드백>")] if "user_feedback" in state else []
    ))

    return {"node_answer": node_answer.content}

def should_continue(state: SearchRagState) -> Literal["계속", "종료"]:
    """웹 검색 계속 여부 결정 함수"""
    print(f"현재 생성 횟수: {state['num_generations']}, 추출된 웹 정보 수: {len(state['processed_info'])}")
    
    # 웹 검색은 정보가 많으므로 조건을 조정
    if state["num_generations"] >= 2:  # 최대 2번 재시도
        return "종료"

    if len(state["processed_info"]) > 0:  # 웹 정보가 1개 이상이면 충분
        return "종료"

    return "계속"

In [ ]:
# 그래프 생성
workflow = StateGraph(SearchRagState)

# 노드 추가 
workflow.add_node("retrieve", retrieve_documents)
workflow.add_node("evaluate_single_document", evaluate_single_document)  # 병렬 워커 노드
workflow.add_node("aggregate_results", aggregate_results)                # 집계 노드
workflow.add_node("rewrite_query", rewrite_query)
workflow.add_node("generate_answer", generate_node_answer)

# 엣지 추가 
workflow.add_edge(START, "retrieve")

# 조건부 엣지로 병렬 워커 생성 (retrieve → 병렬 워커들)
workflow.add_conditional_edges(
    "retrieve",  # 문서 검색 완료 후
    create_evaluation_workers,  # 각 문서에 대해 병렬 워커 생성
    ["evaluate_single_document"]  # 워커 노드들로 분기
)

# 병렬 처리 완료 후 집계 (워커들 → aggregate_results)
workflow.add_edge("evaluate_single_document", "aggregate_results")

# 집계 후 조건부 분기 (aggregate_results → 계속/종료)
workflow.add_conditional_edges(
    "aggregate_results",  # 집계 완료 후
    should_continue,      # 계속할지 결정
    {
        "계속": "rewrite_query",    # 정보 부족 시 쿼리 재작성
        "종료": "generate_answer"   # 충분한 정보 시 답변 생성
    }
)

# 재작성된 쿼리로 다시 검색
workflow.add_edge("rewrite_query", "retrieve")

# 최종 답변 생성 완료
workflow.add_edge("generate_answer", END)

# 그래프 컴파일
search_web_agent = workflow.compile()

# 그래프 시각화
display(Image(search_web_agent.get_graph().draw_mermaid_png()))

In [ ]:
inputs = {"question": "대리인과 아파트 임대차 계약을 체결할 때 주의해야 할 점은 무엇인가요?"}
for output in search_web_agent.stream(inputs):
    for key, value in output.items():
        # 노드 출력
        pprint(f"Node '{key}':")
        pprint(f"Value: {value}", indent=2, width=80, depth=None)
    print("\n----------------------------------------------------------\n")

In [ ]:
# 마크다운 형식으로 출력
from IPython.display import Markdown
display(Markdown(value['node_answer']))

### 4-2. 질문 라우팅 
- 사용자의 질문을 분석하여 적절한 에이전트를 선택 (Adaptive RAG 적용)

In [ ]:
from typing import Annotated
from operator import add

# 딕셔서리 병합을 위한 사용자 정의 리듀서 
def merge_agent_responses(left: dict, right: dict) -> dict:
    """에이전트 응답들을 병합하는 리듀서"""
    if left is None:
        left = {}
    if right is None:
        right = {}
    
    # 딕셔너리 병합
    merged = left.copy()
    merged.update(right)
    return merged

# 상태 정의
class ResearchAgentState(TypedDict):
    question: str
    answers: Annotated[List[str], add]  # 각 에이전트의 답변들
    agent_responses: Annotated[dict, merge_agent_responses]  # 병렬 처리 지원 리듀서
    final_answer: str
    datasources: List[str]
    evaluation_report: Optional[dict]
    iteration_count: Optional[int]
    user_decision: Literal["continue", "stop"]  # 사용자 결정 (계속/종료)
    user_feedback: Optional[str]  # 사용자 피드백

In [ ]:
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from pydantic import BaseModel, Field

# 라우팅 모델 정의
class ToolSelector(BaseModel):
    """Routes the user question to the most appropriate tool."""
    tool: Literal["search_personal", "search_labor", "search_housing", "search_web", "llm_fallback"] = Field(
        description="Select one of the tools, based on the user's question.",
    )

class ToolSelectors(BaseModel):
    """Select the appropriate tools that are suitable for the user question."""
    tools: List[ToolSelector] = Field(
        description="Select one or more tools, based on the user's question.",
    )

# 라우팅 시스템
structured_llm_tool_selector = llm.with_structured_output(ToolSelectors)

# 라우팅 시스템 프롬프트
system = dedent("""You are an AI assistant specializing in routing user questions to the appropriate tools.
Use the following guidelines to select the most suitable tool(s):

## 법률 관련 도구 (Law-specific Tools)
- **search_personal**: 개인정보 보호법의 구체적인 조항, 법적 규정, 의무사항, 처벌 규정 등에 대한 질문
- **search_labor**: 근로기준법의 구체적인 조항, 근로 조건, 임금, 휴가, 해고 등 법적 규정에 대한 질문  
- **search_housing**: 주택임대차보호법의 구체적인 조항, 임대차 계약, 보증금, 임차인 권리 등에 대한 질문

## 웹 검색 도구 (Web Search Tool)
**search_web**을 사용하는 경우:
- 최신 정보나 실시간 데이터가 필요한 질문
- 현재 뉴스, 트렌드, 최근 사건에 대한 질문
- 법률 관련이지만 구체적인 조항보다는 현실적 적용 사례나 판례가 필요한 경우
- 통계, 시장 데이터, 가격 정보 등 변동하는 정보
- 특정 회사, 제품, 서비스에 대한 현재 정보

## LLM 일반 응답 도구 (LLM Fallback Tool)
**llm_fallback**을 사용하는 경우:
- **개념적/이론적 설명**: 일반적인 개념, 정의, 원리 설명 (검색 불필요)
- **창의적 작업**: 글쓰기, 시, 소설, 아이디어 생성 등
- **논리적 추론**: 수학 계산, 논리 문제, 분석적 사고
- **개인적 조언**: 의견, 추천, 개인적 상담 (사실 확인 불필요)
- **일반적 대화**: 인사, 감사, 일상 대화
- **질문이 모호하거나 분류하기 어려운 경우**

Always choose all of the appropriate tools based on the user's question.
If a question is about a law but doesn't seem to be asking about specific legal provisions, include both the relevant law search tool and the search_web tool.
""")

route_prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    MessagesPlaceholder("user_feedback"),
    ("human", "{question}"),
])

question_tool_router = route_prompt | structured_llm_tool_selector

# 테스트 실행
print(question_tool_router.invoke({"question": "근로계약 체결할 때 개인정보 취급 상의 유의사항은 무엇인가요?", "user_feedback": []}))
print(question_tool_router.invoke({"question": "개인정보보호법에서 정한 가명정보의 정의는 무엇인가요?", "user_feedback": []}))
print(question_tool_router.invoke({"question": "대리인과 아파트 임대차 계약을 체결할 때 주의해야 할 점은 무엇인가요?", "user_feedback": []}))
print(question_tool_router.invoke({"question": "오늘 날씨 어때?", "user_feedback": []}))
print(question_tool_router.invoke({"question": "내 이름은 무엇인가요?", "user_feedback": []}))

In [ ]:
def analyze_question_tool_search(state: ResearchAgentState):
    """질문을 분석하여 적절한 도구들을 선택"""
    question = state["question"]
    print(f"도구 선택을 위해 질문 분석 중: {question}")
    
    # 사용자 피드백이 있는 경우, 도구 선택에 반영
    user_feedback = state.get("user_feedback", None)

    result = question_tool_router.invoke({
        "question": question, 
        "user_feedback": [("system", f"***You SHOULD consider the following user feedback when selecting tools:***\n<User feedback>{user_feedback}</User feedback>")] if user_feedback else []
    })

    datasources = [tool.tool for tool in result.tools]
    print(f"선택된 도구들: {datasources}")
    return {"datasources": datasources, "agent_responses": {}}

def route_datasources_tool_search(state: ResearchAgentState) -> List[str]:
    """선택된 데이터 소스들로 라우팅"""
    datasources = set(state['datasources'])
    valid_sources = {"search_personal", "search_labor", "search_housing", "search_web", "llm_fallback"}
    
    if datasources.issubset(valid_sources):
        return list(datasources)
    
    return ["llm_fallback"]  # 유효한 도구 호출이 없는 경우, 기본적으로 LLM 폴백 사용

In [ ]:
# 최종 답변 생성 프롬프트
rag_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert assistant specializing in synthesizing and integrating multiple agent responses to provide comprehensive, accurate answers.

## Core Guidelines

### 1. Information Integration Strategy
- **Synthesize ALL available information** from different agents to create the most complete answer
- **Prioritize accuracy and relevance** over brevity
- **Combine complementary information** from multiple sources to enhance answer quality
- **Preserve the unique value** each agent contributes

### 2. Source Prioritization (in order of reliability)
1. **Legal provisions** (개인정보보호법, 근로기준법, 주택임대차보호법 agents)
2. **Official government sources** (from web search)
3. **Authoritative websites and institutions** 
4. **Recent news and reliable media**
5. **General web sources**

### 3. Content Integration Rules
- **Start with the most authoritative answer** as the foundation
- **Add supplementary information** from other agents to enrich the response
- **Include practical applications** and real-world context when available
- **Mention recent developments** or updates if provided by web search
- **Address different aspects** of the question using information from multiple agents

### 4. Citation Format (Enhanced)
- **Legal provisions**: "법률명 제X조 Y항"
- **Web sources**: "출처명 (URL, 날짜)" 
- **Multiple agents**: "복수 출처 종합"
- **Comparative info**: "A에 따르면 X이며 (출처A), B는 Y로 보완 설명함 (출처B)"

### 5. Answer Structure Template
1. **핵심 답변**: 질문에 대한 직접적이고 명확한 답변
2. **법적 근거**: 관련 법률 조항 및 규정 (해당 시)
3. **실무적 적용**: 현실적인 적용 방법이나 절차
4. **최신 동향**: 최근 변화나 업데이트 사항 (해당 시)
5. **주의사항**: 예외 조건이나 추가 고려사항

### 6. Special Instructions
- **Never discard valuable information** from any agent unless it's clearly incorrect
- **Always attempt to find connections** between different pieces of information
- **Include quantitative data** (dates, amounts, percentages) when available

Only say "제공된 정보로는 충분한 답변을 할 수 없습니다" if NO agents provided relevant information.
"""
    ),
    MessagesPlaceholder("user_feedback"),
    ("human", """Synthesize the following agent responses to provide a comprehensive answer to the user's question.

<Question>
{question}
</Question>

<Agent Responses>
{agent_responses}
</Agent Responses>

Create a comprehensive, well-structured answer that maximizes the value of all provided information."""),
])

# 에이전트 응답 포맷터
def format_agent_responses(agent_responses_dict):
    """에이전트 응답들을 프롬프트에 적합한 형태로 포맷팅"""
    if not agent_responses_dict:
        return "에이전트 응답이 없습니다."
    
    formatted_responses = []
    agent_labels = {
        'search_personal': '🔒 개인정보보호법 전문 에이전트',
        'search_labor': '⚖️ 근로기준법 전문 에이전트',
        'search_housing': '🏠 주택임대차보호법 전문 에이전트',
        'search_web': '🌐 웹 검색 에이전트',
        'llm_fallback': '🤖 일반 지식 에이전트'
    }
    
    for agent_type, response in agent_responses_dict.items():
        if response and response.strip():
            label = agent_labels.get(agent_type, f'📋 {agent_type} 에이전트')
            formatted_responses.append(f"{label}:\n{response.strip()}\n")
    
    return "\n" + "="*50 + "\n".join(formatted_responses) + "="*50

In [ ]:
def personal_rag_node(state: ResearchAgentState) -> ResearchAgentState:
    """개인정보보호법 전문 에이전트"""
    print("--- 개인정보보호법 전문가 에이전트 시작 ---")
    question = state["question"]
    user_feedback = state.get("user_feedback", None)

    answer = personal_law_agent.invoke({
        "question": question, 
        "user_feedback": [("system", f"답변을 생성할 때 다음 사용자 피드백을 고려하세요:\n<피드백>{user_feedback}</피드백>")] if user_feedback else []
    })
    response_text = answer.get("node_answer", "답변을 생성할 수 없습니다.")
    
    # 에이전트별 응답 저장
    current_responses = state.get("agent_responses", {}).copy()
    current_responses["search_personal"] = response_text
    
    return {
        "answers": [response_text],
        "agent_responses": current_responses
    }

def labor_rag_node(state: ResearchAgentState) -> ResearchAgentState:
    """근로기준법 전문 에이전트"""
    print("--- 근로기준법 전문가 에이전트 시작 ---")
    question = state["question"]
    user_feedback = state.get("user_feedback", None)

    answer = labor_law_agent.invoke({
        "question": question, 
        "user_feedback": [("system", f"답변을 생성할 때 다음 사용자 피드백을 고려하세요:\n<피드백>{user_feedback}</피드백>")] if user_feedback else []
    })
    response_text = answer.get("node_answer", "답변을 생성할 수 없습니다.")
    current_responses = state.get("agent_responses", {}).copy()
    current_responses["search_labor"] = response_text
    
    return {
        "answers": [response_text],
        "agent_responses": current_responses
    }

def housing_rag_node(state: ResearchAgentState) -> ResearchAgentState:
    """주택임대차보호법 전문 에이전트"""
    print("--- 주택임대차보호법 전문가 에이전트 시작 ---")
    question = state["question"]
    user_feedback = state.get("user_feedback", None)

    answer = housing_law_agent.invoke({
        "question": question, 
        "user_feedback": [("user", f"답변을 생성할 때 다음 사용자 피드백을 고려하세요:\n<피드백>{user_feedback}</피드백>")] if user_feedback else []
    })
    response_text = answer.get("node_answer", "답변을 생성할 수 없습니다.")

    current_responses = state.get("agent_responses", {}).copy()
    current_responses["search_housing"] = response_text

    return {
        "answers": [response_text],
        "agent_responses": current_responses
    }



def web_rag_node(state: ResearchAgentState) -> ResearchAgentState:
    """웹 검색 전문 에이전트"""
    print("--- 인터넷 검색 전문가 에이전트 시작 ---")
    question = state["question"]
    user_feedback = state.get("user_feedback", None)

    answer = search_web_agent.invoke({
        "question": question, 
        "user_feedback": [("user", f"답변을 생성할 때 다음 사용자 피드백을 고려하세요:\n<피드백>{user_feedback}</피드백>")] if user_feedback else []
    })
    response_text = answer.get("node_answer", "답변을 생성할 수 없습니다.")

    current_responses = state.get("agent_responses", {}).copy()
    current_responses["search_web"] = response_text
    
    return {
        "answers": [response_text],
        "agent_responses": current_responses
    }

def answer_final(state: ResearchAgentState) -> dict:
    """최종 답변 생성 함수"""
    print("---최종 답변 생성---")
    question = state["question"]
    user_feedback = state.get("user_feedback", None)
    # 에이전트 응답들을 가져옴
    agent_responses = state.get("agent_responses", {})
    
    # 에이전트 응답이 있는지 확인
    if not agent_responses:
        return {
            "final_answer": "에이전트로부터 답변을 받지 못했습니다.",
            "question": question
        }
    
    # 에이전트 응답 포맷팅
    formatted_responses = format_agent_responses(agent_responses)
    
    print(f"통합할 에이전트 응답 수: {len(agent_responses)}")
    for agent_type in agent_responses.keys():
        print(f"  - {agent_type}")
    
    # RAG 체인으로 최종 답변 생성
    rag_chain = rag_prompt | llm | StrOutputParser()

    generation = rag_chain.invoke({
        "question": question,
        "agent_responses": formatted_responses,
        "user_feedback": [("system", f"최종 답변을 생성할 때 다음 사용자 피드백을 고려하세요:\n<피드백>{user_feedback}</피드백>")] if user_feedback else []
    })
    print(f"최종 답변 생성 완료: {generation[:100]}...")  # 처음 100자만 출력

    return {
        "final_answer": generation,
        "question": question
    }

# ===== LLM Fallback =====
fallback_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an AI assistant helping with various topics. Follow these guidelines:
1. Provide accurate and helpful information to the best of your ability.
2. Express uncertainty when unsure; avoid speculation.
3. Keep answers concise yet informative.
4. Respond ethically and constructively."""),
    MessagesPlaceholder("user_feedback"),
    ("human", "{question}"),
])

def llm_fallback(state: ResearchAgentState) -> dict:
    """LLM 일반 응답"""
    print("---Fallback 답변---")
    question = state["question"]
    user_feedback = state.get("user_feedback", None)

    llm_chain = fallback_prompt | llm | StrOutputParser()
    generation = llm_chain.invoke({
        "question": question,
        "user_feedback": [("system", f"답변을 생성할 때 다음 사용자 피드백을 고려하세요:\n<피드백>{user_feedback}</피드백>")] if user_feedback else []
    })
    print(f"Fallback 답변 생성 완료: {generation[:100]}...")  # 처음 100자만 출력
    return {
        "final_answer": generation,
        "question": question
    }

In [ ]:
# 노드 정의를 딕셔너리로 관리
nodes = {
    "analyze_question": analyze_question_tool_search,
    "search_personal": personal_rag_node,
    "search_labor": labor_rag_node,
    "search_housing": housing_rag_node,
    "search_web": web_rag_node,
    "generate_answer": answer_final,
    "llm_fallback": llm_fallback
}

# 그래프 생성
search_builder = StateGraph(ResearchAgentState)

# 노드 추가
for node_name, node_func in nodes.items():
    search_builder.add_node(node_name, node_func)

# 엣지 추가
search_builder.add_edge(START, "analyze_question")

# 조건부 엣지 (병렬 처리)
search_builder.add_conditional_edges(
    "analyze_question",
    route_datasources_tool_search,
    ["search_personal", "search_labor", "search_housing", "search_web", "llm_fallback"]
)

# 검색 노드들을 generate_answer에 연결
for node in ["search_personal", "search_labor", "search_housing", "search_web"]:
    search_builder.add_edge(node, "generate_answer")

search_builder.add_edge("generate_answer", END)
search_builder.add_edge("llm_fallback", END)

# 그래프 컴파일
rag_search_graph = search_builder.compile()

# 그래프 시각화
display(Image(rag_search_graph.get_graph().draw_mermaid_png()))

In [ ]:
# 그래프 시각화
display(Image(rag_search_graph.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
# 1개의 도구가 선택된 경우
inputs = {
    "question": "대리인과 아파트 임대차 계약을 체결할 때 주의해야 할 점은 무엇인가요?"
}
for output in rag_search_graph.stream(inputs):
    for key, value in output.items():
        # 노드 출력
        pprint(f"Node '{key}':")
        pprint(f"Value: {value}", indent=2, width=80, depth=None)
    print("\n----------------------------------------------------------\n")

In [ ]:
# 마크다운 형식으로 출력
from IPython.display import Markdown
display(Markdown(value['final_answer']))

In [ ]:
# 2개의 도구가 선택된 경우
inputs = {"question": "근로계약 체결할 때 개인정보 취급 상의 유의사항은 무엇인가요?"}
for output in rag_search_graph.stream(inputs):
    for key, value in output.items():
        # 노드 출력
        pprint(f"Node '{key}':")
        pprint(f"Value: {value}", indent=2, width=80, depth=None)
    print("\n----------------------------------------------------------\n")

In [ ]:
# 마크다운 형식으로 출력
from IPython.display import Markdown
display(Markdown(value['final_answer']))

# 5. 답변 평가 및 확인

### 5-1. 답변을 평가하는 ReAct 에이전트

In [ ]:
from textwrap import dedent

evaluation_prompt = dedent("""
당신은 AI 어시스턴트가 생성한 답변을 평가하는 전문가입니다. 주어진 질문과 답변을 평가하고, 60점 만점으로 점수를 매기세요. 다음 기준을 사용하여 평가하십시오:

1. 정확성 (10점)
2. 관련성 (10점)
3. 완전성 (10점)
4. 인용 정확성 (10점)
5. 명확성과 간결성 (10점)
6. 객관성 (10점)

평가 과정:
1. 주어진 질문과 답변을 주의 깊게 읽으십시오.
2. 필요한 경우, 다음 도구를 사용하여 추가 정보를 수집하세요:
   - web_search: 웹 검색
   - personal_law_search: 개인정보보호법 검색
   - labor_law_search: 근로기준법 검색
   - housing_law_search: 주택임대차보호법 검색

   도구 사용 형식:
   Action: [tool_name]
   Action Input: [input for the tool]

3. 각 기준에 대해 1-10점 사이의 점수를 매기세요.
4. 총점을 계산하세요 (60점 만점).

출력 형식:
{
  "scores": {
    "accuracy": 0,
    "relevance": 0,
    "completeness": 0,
    "citation_accuracy": 0,
    "clarity_conciseness": 0,
    "objectivity": 0
  },
  "total_score": 0,
  "brief_evaluation": "간단한 평가 설명"
}

최종 출력에는 각 기준의 점수, 총점, 그리고 간단한 평가 설명만 포함하세요.
""")

In [ ]:
tools

In [ ]:
from langgraph.prebuilt import create_react_agent
from IPython.display import Image, display

# 그래프 생성 
answer_reviewer = create_react_agent(
    ChatOpenAI(model="gpt-4.1"), 
    tools=tools, 
    prompt=evaluation_prompt,
    )

# 그래프 출력
display(Image(answer_reviewer.get_graph().draw_mermaid_png()))

In [ ]:
# 그래프 실행
from langchain_core.messages import HumanMessage

messages = [HumanMessage(content=f"""<질문>\n{value['question']}\n</질문>\n<답변>\n{value['final_answer']}\n</답변>""")]
response = answer_reviewer.invoke({"messages": messages})
for m in response['messages']:
    m.pretty_print()

In [ ]:
pprint(json.loads(m.content))

### 5-2. 답변 평가 내용을 확인하는 HITL 추가

In [ ]:
from langgraph.types import interrupt, Command
from langchain_core.messages import HumanMessage

# 답변 평가하는 노드를 추가
def evaluate_answer_node(state:ResearchAgentState):
    question = state["question"]
    final_answer = state["final_answer"]

    messages = [HumanMessage(content=f"""[질문]\n{question}\n[답변]\n{final_answer}""")]
    response = answer_reviewer.invoke({"messages": messages})
    response_dict = json.loads(response['messages'][-1].content)

    return {"evaluation_report": response_dict, "question": question, "final_answer": final_answer}


# interrupt를 사용한 HITL 노드
def human_review_node(state: ResearchAgentState):
    """interrupt를 사용한 사람의 검토 노드"""
    
    # 사용자에게 표시할 정보 구성
    review_data = {
        "question": state["question"],
        "final_answer": state["final_answer"],
        "evaluation_report": state["evaluation_report"],
        "total_score": state["evaluation_report"].get("total_score", 0),
        "brief_evaluation": state["evaluation_report"].get("brief_evaluation", ""),
        "instruction": "답변을 검토해주세요 (approve/reject):"
    }
    
    print(f"\n현재 답변: {state['final_answer']}")
    print(f"평가 결과: 총점 {state['evaluation_report']['total_score']}/60")
    print(f"{state['evaluation_report']['brief_evaluation']}")
    
    # interrupt를 사용하여 사용자 입력 대기
    user_input = interrupt(value=review_data)
    
    # 사용자 입력 처리 개선
    if isinstance(user_input, dict):
        decision = user_input.get("decision", "").lower()
        feedback = user_input.get("feedback", "")
    elif isinstance(user_input, str):
        if user_input.lower() in ['y', 'yes', 'approve', 'approved', 'ok']:
            decision = "approved"
            feedback = ""
        else:
            decision = "rejected"
            feedback = user_input if user_input else "답변이 거부되었습니다."
    else:
        # 빈 입력이나 None인 경우 기본값 설정
        decision = "rejected"
        feedback = "입력이 없어 자동으로 거부되었습니다."
    
    return {
        "user_decision": decision,
        "user_feedback": feedback,
        "iteration_count": state.get("iteration_count", 0) + 1
    }


# 조건부 라우팅 함수
def route_after_human_review(state: ResearchAgentState) -> Literal["approved", "rejected"]:
    """human_review 노드 후 라우팅 결정"""
    user_decision = state.get("user_decision", "").lower()
    
    if user_decision in ["approved", "approve", "y", "yes"]:
        return "approved"
    else:
        return "rejected"

# 승인된 경우 처리하는 노드
def approved_node(state: ResearchAgentState):
    """답변이 승인된 경우의 최종 처리"""
    print("✅ 답변이 승인되었습니다!")
    return {
        "user_decision": "approved",
        "final_answer": state["final_answer"]
    }

# 거부된 경우 처리하는 노드 
def rejected_node(state: ResearchAgentState):
    """답변이 거부된 경우의 처리"""
    print(f"❌ 답변이 거부되었습니다. 피드백: {state.get('user_feedback', '')}")
    
    # 최대 반복 횟수 체크 (예: 3회)
    if state.get("iteration_count", 0) >= 3:
        print("최대 반복 횟수에 도달했습니다. 현재 답변으로 진행합니다.")
        return {"final_answer": f"[최대 수정 후 자동 승인] {state['final_answer']}"}
    
    # 답변을 수정하는 로직을 여기에 추가할 수 있습니다
    # 예: LLM에게 피드백을 전달하여 답변 재생성
    
    return {
        "user_feedback": state.get("user_feedback", ""),
        "iteration_count": state.get("iteration_count", 0)
    }


In [ ]:
from langgraph.checkpoint.memory import InMemorySaver

# 노드 정의
nodes = {
    "analyze_question": analyze_question_tool_search,
    "search_personal": personal_rag_node,
    "search_labor": labor_rag_node,
    "search_housing": housing_rag_node,
    "search_web": web_rag_node,
    "generate_answer": answer_final,
    "llm_fallback": llm_fallback,
    "evaluate_answer": evaluate_answer_node,
    "human_review": human_review_node,
    "approved": approved_node,
    "rejected": rejected_node,  # 거부 처리 노드 추가
}

# StateGraph 생성
search_builder = StateGraph(ResearchAgentState)

# 노드 추가
for node_name, node_func in nodes.items():
    search_builder.add_node(node_name, node_func)

# 엣지 추가
search_builder.add_edge(START, "analyze_question")

# 조건부 엣지 - 데이터 소스 라우팅
search_builder.add_conditional_edges(
    "analyze_question",
    route_datasources_tool_search,
    ["search_personal", "search_labor", "search_housing", "search_web", "llm_fallback"]
)

# 검색 노드들을 generate_answer에 연결
for node in ["search_personal", "search_labor", "search_housing", "search_web"]:
    search_builder.add_edge(node, "generate_answer")

search_builder.add_edge("generate_answer", "evaluate_answer")
search_builder.add_edge("evaluate_answer", "human_review")

# HITL 결과에 따른 조건부 엣지 - 수정된 부분
search_builder.add_conditional_edges(
    "human_review",
    route_after_human_review,
    {
        "approved": "approved",  # approved 노드로 이동
        "rejected": "rejected"   # rejected 노드로 이동
    }
)

# 최종 엣지들
search_builder.add_edge("approved", END)
search_builder.add_edge("rejected", "analyze_question")  # 거부 시 처음부터 다시 시작
search_builder.add_edge("llm_fallback", END)

# 메모리 추가 (interrupt 사용을 위해 필수)
checkpointer = InMemorySaver()

# 그래프 컴파일
legal_rag_agent = search_builder.compile(checkpointer=checkpointer)

# 그래프 시각화
display(Image(legal_rag_agent.get_graph().draw_mermaid_png()))

In [ ]:
# 질문 입력
inputs = {"question": "대리인과 아파트 임대차 계약을 체결할 때 주의해야 할 점은 무엇인가요?"}

# 스레드 설정
thread_config = {
    "configurable": {
        "thread_id": "legal_rag_thread",
    }
}
for output in legal_rag_agent.stream(inputs, config=thread_config):
    for key, value in output.items():
        # 노드 출력
        pprint(f"Node '{key}':")
        pprint(f"Value: {value}", indent=2, width=80, depth=None)
    print("\n----------------------------------------------------------\n")

In [ ]:
# 현재 중단된 노드 확인
key

In [ ]:
# 중단점의 상태 값 확인
pprint(value[0].value)

- **사용자 개입 (HITL): 거부**

    - 사용자가 승인을 해주지 않는 경우, 피드백을 통해 다시 생성
    - **Command** 함수로 사용자 피드백 전달

In [ ]:
from langgraph.types import Command

# 사용자 피드백 제공 (거부)
human_feedback = {
    "decision": "rejected",
    "feedback": "답변에 대한 출처가 충분히 구체적이지 않습니다. 인용된 법조항에 대한 구체적 조문과 더 많은 세부사항이 필요합니다."
}

# 사용자 입력으로 그래프 재개
for output in legal_rag_agent.stream(Command(resume=human_feedback), config=thread_config):
    for key, value in output.items():
        # 노드 출력
        pprint(f"Node '{key}':")
        pprint(f"Value: {value}", indent=2, width=80, depth=None)
    print("\n----------------------------------------------------------\n")

In [ ]:
# 현재 상태 - 그래프 체크포인트 확인
current_state = legal_rag_agent.get_state(thread_config)

print(f"\n현재 상태: {current_state.values}")
print(f"대기 중인 interrupt: {current_state.tasks}")

In [ ]:
# 대기 중인 interrupt 확인
task = current_state.tasks[0]
print(task.id)
print(task.name)
pprint(task.interrupts[0].value)

In [ ]:
# 다음에 실행될 노드를 확인 
current_state.next

- **사용자 개입 (HITL): 승인**

- 사용자가 '승인'을 해서 최종 답변을 생성하고 종료

In [ ]:
from langgraph.types import Command

# 사용자 피드백 제공 (승인 문자열을 사용)
human_feedback = "approved"

# 사용자 입력으로 그래프 재개
for output in legal_rag_agent.stream(Command(resume=human_feedback), config=thread_config):
    for key, value in output.items():
        # 노드 출력
        pprint(f"Node '{key}':")
        pprint(f"Value: {value}", indent=2, width=80, depth=None)
    print("\n----------------------------------------------------------\n")

In [ ]:
# 현재 상태 - 그래프 체크포인트 확인
current_state = legal_rag_agent.get_state(thread_config)

print(f"\n현재 상태: {current_state.values}")
print(f"대기 중인 interrupt: {current_state.tasks}")

In [ ]:
# 대기 중인 interrupt 확인
current_state.tasks

In [ ]:
# 최종 상태 확인
current_state.values

In [ ]:
# 다음에 실행될 노드를 확인 
current_state.next

In [ ]:
# 마크다운 형식으로 최종 답변 출력
from IPython.display import Markdown
final_answer = current_state.values.get("final_answer", "답변이 없습니다.")
Markdown(f"### 최종 답변\n{final_answer}")

# 6. Gradio 챗봇

In [ ]:
import gradio as gr
import uuid
from typing import List, Tuple, Dict, Any, Optional
from langgraph.types import Command
import json

# 예시 질문들
example_questions = [
    "개인정보 유출 시 기업이 취해야 할 법적 조치는 무엇인가요?",
    "전월세 계약 갱신 요구권의 행사 기간과 조건은 어떻게 되나요?",
    "퇴직금 지급 기준과 계산 방법은 어떻게 되나요?",
]

# 개선된 ChatBot 클래스
class LegalRAGChatBot:
    def __init__(self, graph):
        self.graph = graph
        self.thread_id = str(uuid.uuid4())
        self.waiting_for_approval = False
        self.current_interrupt_data = None
        print(f"ChatBot 초기화 완료 - Thread ID: {self.thread_id}")
    
    def get_thread_config(self):
        """스레드 설정 반환"""
        return {"configurable": {"thread_id": self.thread_id}}
    
    def format_review_message(self, interrupt_data):
        """검토 메시지 포맷팅"""
        if not interrupt_data:
            return "❌ 검토 데이터를 가져올 수 없습니다."
        
        final_answer = interrupt_data.get("final_answer", "답변 없음")
        evaluation_report = interrupt_data.get("evaluation_report", {})
        
        total_score = evaluation_report.get("total_score", 0)
        brief_evaluation = evaluation_report.get("brief_evaluation", "평가 없음")
        
        return f"""**🔍 답변 검토가 필요합니다**

**생성된 답변:**
{final_answer}

**자동 평가 결과:**
• **총점:** {total_score}/60점
• **평가:** {brief_evaluation}

---
**다음 중 하나를 선택해주세요:**
• **'y'** 또는 **'승인'** → 답변 승인
• **'n'** 또는 **'수정'** → 답변 수정 요청
• **'n: 수정의견'** → 구체적인 수정의견과 함께 수정 요청
  
**예시:** `n: 더 구체적인 사례와 벌금 기준을 추가해주세요`"""

    def process_message(self, message: str) -> str:
        """메시지 처리 메인 로직"""
        try:
            if not self.waiting_for_approval:
                # 새로운 질문 처리
                return self._handle_new_question(message)
            else:
                # 승인/거부 처리
                return self._handle_approval_decision(message)
                
        except Exception as e:
            self.waiting_for_approval = False
            self.current_interrupt_data = None
            print(f"전체 처리 오류: {e}")
            return f"❌ **오류가 발생했습니다:** {str(e)}\n\n새로운 질문을 입력해주세요."

    def _handle_new_question(self, message: str) -> str:
        """새로운 질문 처리"""
        print(f"새 질문 처리: {message}")
        
        # 초기 상태 구성
        initial_state = {
            "question": message,
            "analysis_result": {},
            "search_results": {},
            "final_answer": "",
            "evaluation_report": {},
            "user_decision": "",
            "user_feedback": "",
            "iteration_count": 0
        }
        
        try:
            config = self.get_thread_config()
            
            # graph.stream()을 사용하여 interrupt 발생 감지
            result_chunks = []
            interrupt_detected = False
            
            for chunk in self.graph.stream(initial_state, config=config):
                print(f"스트림 청크: {chunk}")
                result_chunks.append(chunk)
                
                # interrupt 발생 여부 확인
                if '__interrupt__' in chunk:
                    print("Interrupt 감지됨!")
                    interrupt_detected = True
                    
                    # interrupt 정보 추출
                    interrupts = chunk['__interrupt__']
                    if interrupts and len(interrupts) > 0:
                        interrupt_info = interrupts[0]  # 첫 번째 interrupt 사용
                        self.current_interrupt_data = interrupt_info.value
                        
                        print(f"Interrupt 데이터: {self.current_interrupt_data}")
                        
                        self.waiting_for_approval = True
                        return self.format_review_message(self.current_interrupt_data)
                    else:
                        return "❌ interrupt 데이터가 비어있습니다."
            
            # interrupt가 발생하지 않고 정상 완료된 경우
            if not interrupt_detected:
                # 마지막 상태에서 final_answer 추출
                final_state = self.graph.get_state(config)
                final_answer = final_state.values.get('final_answer', '답변을 생성할 수 없습니다.')
                return f"✅ **답변 완료**\n\n{final_answer}"
            
        except Exception as e:
            print(f"새 질문 처리 중 오류: {e}")
            raise e

    def _handle_approval_decision(self, message: str) -> str:
        """승인/거부 결정 처리"""
        print(f"승인/거부 처리: {message}")
        
        # 사용자 입력 분석
        user_input = message.strip().lower()
        
        # 결정과 피드백 분석
        if user_input in ['y', 'yes', 'approved', 'approve', '승인', '예', 'ok', '좋아']:
            decision = "approved"
            feedback = ""
        elif user_input.startswith('n:'):
            decision = "rejected"
            feedback = message[2:].strip()  # 'n:' 이후의 텍스트를 피드백으로 사용
        else:
            decision = "rejected"
            feedback = message if message not in ['n', 'no', 'reject', '수정', '거부'] else "답변 수정이 필요합니다."
        
        print(f"사용자 결정: {decision}, 피드백: {feedback}")
        
        try:
            config = self.get_thread_config()
            
            # Command를 사용하여 interrupt 지점에서 재개
            resume_data = {
                "decision": decision,
                "feedback": feedback
            }
            
            result_chunks = []
            interrupt_detected = False
            
            # Command(resume=)를 사용하여 그래프 재개
            for chunk in self.graph.stream(Command(resume=resume_data), config=config):
                print(f"재개 스트림 청크: {chunk}")
                result_chunks.append(chunk)
                
                # 또 다른 interrupt 발생 여부 확인 (수정된 답변에 대한 재검토)
                if '__interrupt__' in chunk:
                    print("재개 중 또 다른 interrupt 감지됨!")
                    interrupt_detected = True
                    
                    interrupts = chunk['__interrupt__']
                    if interrupts and len(interrupts) > 0:
                        interrupt_info = interrupts[0]
                        self.current_interrupt_data = interrupt_info.value
                        
                        # waiting_for_approval은 이미 True 상태 유지
                        response = self.format_review_message(self.current_interrupt_data)
                        # 수정된 답변임을 표시
                        response = response.replace("🔍 답변 검토가 필요합니다", "🔄 수정된 답변 검토")
                        response = response.replace("생성된 답변:", "수정된 답변:")
                        
                        return response
                    else:
                        self.waiting_for_approval = False
                        return "❌ 수정된 답변의 interrupt 데이터가 비어있습니다."
            
            # interrupt가 발생하지 않고 완료된 경우
            if not interrupt_detected:
                self.waiting_for_approval = False
                self.current_interrupt_data = None
                
                # 최종 상태에서 답변 추출
                final_state = self.graph.get_state(config)
                final_answer = final_state.values.get('final_answer', '최종 답변을 가져올 수 없습니다.')
                
                if decision == "approved":
                    return f"✅ **답변이 승인되었습니다!**\n\n{final_answer}"
                else:
                    return f"🔄 **답변 수정 완료**\n\n{final_answer}"
                
        except Exception as e:
            print(f"승인/거부 처리 중 오류: {e}")
            self.waiting_for_approval = False
            raise e

    def chat(self, message: str, history: List[Tuple[str, str]]) -> str:
        """Gradio ChatInterface용 메인 함수"""
        print(f"\n=== 채팅 호출 ===")
        print(f"Thread ID: {self.thread_id}")
        print(f"대기 상태: {self.waiting_for_approval}")
        print(f"메시지: {message}")
        print(f"히스토리 길이: {len(history)}")
        
        try:
            response = self.process_message(message)
            print(f"응답 길이: {len(response)}")
            return response
        except Exception as e:
            print(f"채팅 처리 중 오류: {e}")
            self.waiting_for_approval = False
            self.current_interrupt_data = None
            return f"❌ **처리 중 오류가 발생했습니다:** {str(e)}\n\n새로운 질문을 입력해주세요."

    def reset_session(self):
        """새로운 세션 시작"""
        old_thread_id = self.thread_id
        self.thread_id = str(uuid.uuid4())
        self.waiting_for_approval = False
        self.current_interrupt_data = None
        print(f"세션 리셋: {old_thread_id} → {self.thread_id}")
        return "🔄 새로운 대화를 시작합니다."

    def get_current_state(self):
        """현재 그래프 상태 반환 (디버깅용)"""
        try:
            config = self.get_thread_config()
            state = self.graph.get_state(config)
            return {
                "values": state.values,
                "tasks": [task.__dict__ for task in state.tasks] if state.tasks else [],
                "waiting_for_approval": self.waiting_for_approval,
                "current_interrupt_data": self.current_interrupt_data
            }
        except Exception as e:
            return {"error": str(e)}

# Gradio 인터페이스 생성 함수
def create_gradio_interface(chatbot):
    """Gradio 인터페이스를 생성하는 함수"""
    
    with gr.Blocks(
        title="생활법률 AI 어시스턴트 (HITL)", 
        theme=gr.themes.Soft(),
        css="""
        .chat-container { max-width: 1000px; margin: 0 auto; }
        .example-btn { margin: 2px; }
        """,
        analytics_enabled=False,
    ) as demo:
        
        gr.Markdown("""
        # 🏛️ 생활법률 AI 어시스턴트
        
        **주택임대차보호법**, **근로기준법**, **개인정보보호법** 관련 질문에 답변해 드립니다.
        
        """)
        
        # 메인 채팅 인터페이스
        chatinterface = gr.ChatInterface(
            fn=chatbot.chat,
            examples=example_questions,
            title="",
            description="",
            type="messages",
            textbox=gr.Textbox(
                placeholder="법률 질문을 입력하세요 (예: 퇴직금 계산 방법은?)",
                container=False,
                scale=7
            ),
            analytics_enabled=False,
        )
        
    return demo


# ChatBot 생성
chatbot = LegalRAGChatBot(legal_rag_agent)

# Gradio 인터페이스 생성 및 실행
demo = create_gradio_interface(chatbot)
demo.launch(debug=True, share=False)